# INTRODUCTION

# Data Pre-Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# POLICIES BY STATE

## DATA EXPLORATION

In [4]:
df = pd.read_excel('COVID19_US_state_policy_database.xlsx')
df.head()
df.shape

,COVID-19 US state policy database (CUSP),Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,Last date of comprehensive review: 7/14/2020,NaN,NaN
3,Link to data in xlsx format:,https://github.com/KristenNocka/COVID-19-US-St...,NaN
4,Table of Contents,NaN,NaN


(179, 3)

In [5]:
xls_file = pd.ExcelFile('COVID19_US_state_policy_database.xlsx')
xls_file

### Load data from each excel sheet

In [6]:
sheet_names=xls_file.sheet_names
sheet_names

['Information',
 'State of Emergency',
 'Physical Distance Closures',
 'Stay at Home',
 'Reopening',
 '2nd Closures',
 'Face Masks',
 'Quarantine Rules',
 'Alcohol and Firearms',
 'Housing',
 'Unemployment Insurance',
 'Food Security',
 'Healthcare Delivery',
 'Racial Disparities',
 'Incarcerated Individuals',
 'Substance Use Disorder Policies',
 'Pre-Covid Policies',
 'State Characteristics',
 'NotesDetails']

#### State of Emergency

In [7]:
State_of_Emergency = xls_file.parse('State of Emergency')
State_of_Emergency.shape
State_of_Emergency.dropna(inplace=True)
State_of_Emergency.tail()

(52, 4)

,State,State Abbreviation,State FIPS Code,State of emergency
46,Virginia,VA,51.0,2020-03-12 00:00:00
47,Washington,WA,53.0,2020-02-29 00:00:00
48,West Virginia,WV,54.0,2020-03-16 00:00:00
49,Wisconsin,WI,55.0,2020-03-12 00:00:00
50,Wyoming,WY,56.0,2020-03-13 00:00:00


In [8]:
State_of_Emergency['State of emergency'] = pd.to_datetime(State_of_Emergency['State of emergency'], errors='coerce')

#### Pre-Covid Policies

In [9]:
Pre_Covid_Policies = xls_file.parse('Pre-Covid Policies')
Pre_Covid_Policies.shape
Pre_Covid_Policies.dropna(inplace =True)
Pre_Covid_Policies.tail()

(52, 5)

,State,State Abbreviation,State FIPS Code,Paid sick leave,Medicaid Expansion
46,Virginia,VA,51.0,0,1
47,Washington,WA,53.0,1,1
48,West Virginia,WV,54.0,0,1
49,Wisconsin,WI,55.0,0,0
50,Wyoming,WY,56.0,0,0


#### Food Security

In [10]:
Food_Security= xls_file.parse('Food Security')
Food_Security.dropna(inplace=True)
Food_Security.shape
Food_Security.tail()

(51, 7)

,State,State Abbreviation,State FIPS Code,SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver
46,Virginia,VA,51.0,2020-03-24 00:00:00,2020-04-25 00:00:00,2020-04-13 00:00:00,1
47,Washington,WA,53.0,2020-03-27 00:00:00,2020-05-22 00:00:00,0,1
48,West Virginia,WV,54.0,2020-03-27 00:00:00,2020-04-30 00:00:00,0,1
49,Wisconsin,WI,55.0,2020-04-01 00:00:00,2020-04-22 00:00:00,0,1
50,Wyoming,WY,56.0,2020-04-01 00:00:00,2020-04-16 00:00:00,0,0


In [11]:
# To standardise the policy date, this function will subtract the dates from the state of emergency date.All dates will reflect the 
# number of days the policy was made after the state of emergency date
def change_date(df,list):
    df = pd.merge(df, State_of_Emergency, on=['State','State FIPS Code','State Abbreviation'],how='left')
    for col in list:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        df[col] = (df[col] - df['State of emergency']).dt.days
    df.fillna(value = 0 , inplace = True)  
    return df

In [12]:
Food_Security.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'SNAP Waiver-Emergency Allotments to Current SNAP Households',
       'SNAP Waiver-Pandemic EBT',
       'SNAP Waiver-Temporary Suspension of Claims Collection',
       '2020 Q1 SNAP ABAWD Time Limit Waiver'],
      dtype='object')

In [13]:
Food_Security_dates = ['SNAP Waiver-Emergency Allotments to Current SNAP Households','SNAP Waiver-Pandemic EBT',
       'SNAP Waiver-Temporary Suspension of Claims Collection']

In [14]:
Food_Security_dates_updated =change_date(Food_Security,Food_Security_dates)
Food_Security_dates_updated.tail()

,State,State Abbreviation,State FIPS Code,SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,State of emergency
46,Virginia,VA,51.0,12,44.0,32.0,1,2020-03-12
47,Washington,WA,53.0,27,83.0,0.0,1,2020-02-29
48,West Virginia,WV,54.0,11,45.0,0.0,1,2020-03-16
49,Wisconsin,WI,55.0,20,41.0,0.0,1,2020-03-12
50,Wyoming,WY,56.0,19,34.0,0.0,0,2020-03-13


#### Stay at Home

In [18]:
Stay_at_Home = xls_file.parse('Stay at Home')
Stay_at_Home.dropna(inplace=True)
Stay_at_Home.tail()
Stay_at_Home.shape

,State,State Abbreviation,State FIPS Code,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*
46,Virginia,VA,51.0,2020-03-30 00:00:00,0
47,Washington,WA,53.0,2020-03-23 00:00:00,0
48,West Virginia,WV,54.0,2020-03-24 00:00:00,1
49,Wisconsin,WI,55.0,2020-03-25 00:00:00,0
50,Wyoming,WY,56.0,0,0


(51, 5)

In [19]:
Stay_at_Home.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Stay at home/ shelter in place',
       'Religious Gatherings Exempt Without Clear Social Distance Mandate*'],
      dtype='object')

In [20]:
Stay_at_Home_dates =['Stay at home/ shelter in place']
Stay_at_Home_dates_updated= change_date(Stay_at_Home,Stay_at_Home_dates)      

#### Physical Distance Closures

In [21]:
Physical_Distance_Closures = xls_file.parse('Physical Distance Closures')
Physical_Distance_Closures.dropna(inplace=True)
Physical_Distance_Closures.shape
Physical_Distance_Closures.tail()

(51, 11)

,State,State Abbreviation,State FIPS Code,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars
46,Virginia,VA,51.0,2020-03-16 00:00:00,0,0,0,2020-03-25 00:00:00,2020-03-25 00:00:00,2020-03-25 00:00:00,2020-03-24 00:00:00
47,Washington,WA,53.0,2020-03-17 00:00:00,0,2020-03-16 00:00:00,2020-03-25 00:00:00,2020-03-16 00:00:00,2020-03-16 00:00:00,2020-03-16 00:00:00,2020-03-16 00:00:00
48,West Virginia,WV,54.0,2020-03-16 00:00:00,2020-03-26 00:00:00,0,2020-03-24 00:00:00,2020-03-18 00:00:00,2020-03-19 00:00:00,2020-03-24 00:00:00,2020-03-18 00:00:00
49,Wisconsin,WI,55.0,2020-03-18 00:00:00,0,0,2020-03-25 00:00:00,2020-03-17 00:00:00,2020-03-25 00:00:00,2020-03-25 00:00:00,2020-03-17 00:00:00
50,Wyoming,WY,56.0,2020-03-19 00:00:00,2020-03-19 00:00:00,0,0,2020-03-19 00:00:00,2020-03-19 00:00:00,2020-03-19 00:00:00,2020-03-19 00:00:00


In [22]:
Physical_Distance_Closures.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Date closed K-12 schools', 'Closed day cares',
       'Date banned visitors to nursing homes',
       'Closed non-essential businesses', 'Closed restaurants except take out',
       'Closed gyms', 'Closed movie theaters', 'Closed Bars'],
      dtype='object')

In [23]:
Physical_Distance_Closures_dates=['Date closed K-12 schools', 'Closed day cares','Date banned visitors to nursing homes','Closed non-essential businesses', 'Closed restaurants except take out',
       'Closed gyms', 'Closed movie theaters', 'Closed Bars']

In [24]:
Physical_Distance_Closures_dates_updated =change_date(Physical_Distance_Closures,Physical_Distance_Closures_dates)

#### Reopening

In [25]:
Reopening=xls_file.parse('Reopening')
Reopening.dropna(inplace=True)
Reopening.shape
Reopening.tail()

(51, 16)

,State,State Abbreviation,State FIPS Code,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures
46,Virginia,VA,51.0,2020-06-05 00:00:00,2020-05-29 00:00:00,2020-05-29 00:00:00,1.0,2020-06-12 00:00:00,2020-07-01 00:00:00,2020-05-15 00:00:00,0,2020-05-15 00:00:00,2020-05-28 00:00:00,2020-06-12 00:00:00,0,2020-05-01 00:00:00
47,Washington,WA,53.0,2020-06-01 00:00:00,2020-05-05 00:00:00,0,0.0,0,0,0,0,2020-05-27 00:00:00,0,0,0,2020-04-29 00:00:00
48,West Virginia,WV,54.0,2020-05-04 00:00:00,2020-05-04 00:00:00,2020-05-04 00:00:00,1.0,2020-05-18 00:00:00,2020-06-05 00:00:00,2020-05-04 00:00:00,0,0,2020-05-21 00:00:00,2020-05-26 00:00:00,2020-04-30 00:00:00,2020-04-20 00:00:00
49,Wisconsin,WI,55.0,2020-05-13 00:00:00,2020-05-11 00:00:00,2020-05-13 00:00:00,0.0,2020-05-13 00:00:00,2020-05-13 00:00:00,2020-05-13 00:00:00,0,2020-05-13 00:00:00,2020-05-11 00:00:00,2020-05-13 00:00:00,0,0
50,Wyoming,WY,56.0,0,2020-05-01 00:00:00,2020-05-15 00:00:00,0.0,2020-05-01 00:00:00,2020-05-15 00:00:00,2020-05-01 00:00:00,0,2020-06-15 00:00:00,0,2020-05-15 00:00:00,2020-04-28 00:00:00,0


In [26]:
Reopening.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'End/relax stay at home/shelter in place',
       'Began to reopen businesses statewide', 'Reopen restaurants',
       'Initially reopen restaurants for outdoor dining only', 'Reopened gyms',
       'Reopened movie theaters', 'Reopened hair salons/barber shops',
       'Reopen non-essential construction', 'Reopen Religious gatherings',
       'Reopen non-essential retail', 'Reopen bars', 'Reopen Childcare',
       'Resumed elective medical procedures'],
      dtype='object')

In [27]:
Reopening_dates =['End/relax stay at home/shelter in place',
       'Began to reopen businesses statewide', 'Reopen restaurants',
       'Reopened gyms','Reopen non-essential construction',
       'Reopened movie theaters', 'Reopened hair salons/barber shops',
       'Reopen Religious gatherings',
       'Reopen non-essential retail', 'Reopen bars', 'Reopen Childcare',
       'Resumed elective medical procedures']
Reopening_dates_updated=change_date(Reopening, Reopening_dates)

#### Face Masks

In [28]:
Face_Masks=xls_file.parse('Face Masks')
Face_Masks.dropna(inplace=True)
Face_Masks.shape
Face_Masks.tail()

(51, 8)

,State,State Abbreviation,State FIPS Code,Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses
46,Virginia,VA,51.0,2020-05-29 00:00:00,0,1,0,2020-05-29 00:00:00
47,Washington,WA,53.0,2020-06-26 00:00:00,0,0,1,2020-05-04 00:00:00
48,West Virginia,WV,54.0,2020-07-07 00:00:00,0,0,1,2020-05-04 00:00:00
49,Wisconsin,WI,55.0,0,0,0,0,0
50,Wyoming,WY,56.0,0,0,0,0,2020-05-01 00:00:00


In [29]:
Face_Masks.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Mandate face mask use by all individuals in public spaces',
       'Face mask mandate enforced by fines',
       'Face mask mandate enforced by criminal charge/citation',
       'No legal enforcement of face mask mandate',
       'Mandate face mask use by employees in public-facing businesses'],
      dtype='object')

In [30]:
Face_Masks_dates= ['Mandate face mask use by all individuals in public spaces',
                   'Mandate face mask use by employees in public-facing businesses']
Face_Masks_dates_updated=change_date(Face_Masks,Face_Masks_dates)

#### Second Closures

In [31]:
Second_Closures=xls_file.parse('2nd Closures')
Second_Closures.dropna(inplace=True)
Second_Closures.shape
Second_Closures.tail()

(51, 8)

,State,State Abbreviation,State FIPS Code,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide)
46,Virginia,VA,51.0,0,0,0,0,0
47,Washington,WA,53.0,0,0,0,0,0
48,West Virginia,WV,54.0,2020-07-14 00:00:00,0,0,0,0
49,Wisconsin,WI,55.0,0,0,0,0,0
50,Wyoming,WY,56.0,0,0,0,0,0


In [32]:
Second_Closures.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Begin to Re-Close Bars', 'Re-Close Bars (statewide)',
       'Re-Close Movie Theaters (statewide)', 'Re-Close Gyms (statewide)',
       'Re-Close Indoor Dining (Statewide)'],
      dtype='object')

In [33]:
Second_Closures_dates = ['Begin to Re-Close Bars', 'Re-Close Bars (statewide)',
       'Re-Close Movie Theaters (statewide)', 'Re-Close Gyms (statewide)',
       'Re-Close Indoor Dining (Statewide)']
Second_Closures_dates_updated = change_date(Second_Closures,Second_Closures_dates)

#### Quarantine Rules

In [34]:
Quarantine_Rules=xls_file.parse('Quarantine Rules')
Quarantine_Rules.dropna(inplace=True)
Quarantine_Rules.shape
Quarantine_Rules.tail()

(51, 5)

,State,State Abbreviation,State FIPS Code,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state
46,Virginia,VA,51.0,0,0
47,Washington,WA,53.0,0,0
48,West Virginia,WV,54.0,2020-03-31 00:00:00,0
49,Wisconsin,WI,55.0,0,0
50,Wyoming,WY,56.0,0,2020-04-03 00:00:00


In [35]:
Quarantine_Rules.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Mandate quarantine for those entering the state from specific states',
       'Mandate quarantine for all individuals entering the state from another state'],
      dtype='object')

In [36]:
Quarantine_Rules_dates=['Mandate quarantine for those entering the state from specific states',
       'Mandate quarantine for all individuals entering the state from another state']
Quarantine_Rules_dates_updated=change_date(Quarantine_Rules,Quarantine_Rules_dates)

#### Alcohol and Firearms

In [37]:
Alcohol_and_Firearms=xls_file.parse('Alcohol and Firearms')
Alcohol_and_Firearms.dropna(inplace=True)
Alcohol_and_Firearms.shape
Alcohol_and_Firearms.tail()

(51, 7)

,State,State Abbreviation,State FIPS Code,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open
46,Virginia,VA,51.0,1,2020-03-16 00:00:00,2020-03-19 00:00:00,1
47,Washington,WA,53.0,1,2020-03-24 00:00:00,2020-03-24 00:00:00,0
48,West Virginia,WV,54.0,1,2020-03-18 00:00:00,2020-03-18 00:00:00,1
49,Wisconsin,WI,55.0,1,2020-03-20 00:00:00,0,1
50,Wyoming,WY,56.0,1,2020-03-30 00:00:00,0,1


In [38]:
Alcohol_and_Firearms.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Alcohol/Liquor Stores Open',
       'Allow restaurants to sell takeout alcohol ',
       'Allow restaurants to deliver alcohol', 'Keep Firearms Sellers Open'],
      dtype='object')

In [39]:
Alcohol_and_Firearms_dates= ['Allow restaurants to sell takeout alcohol ',
       'Allow restaurants to deliver alcohol']
Alcohol_and_Firearms_dates_updated= change_date(Alcohol_and_Firearms,Alcohol_and_Firearms_dates)

#### Housing

In [40]:
Housing=xls_file.parse('Housing')
Housing.dropna(inplace=True)
Housing.shape
Housing.tail()

(51, 9)

,State,State Abbreviation,State FIPS Code,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium
46,Virginia,VA,51.0,0,2020-06-08 00:00:00,0,2020-03-16 00:00:00,0,2020-06-28 00:00:00
47,Washington,WA,53.0,2020-03-18 00:00:00,2020-03-18 00:00:00,0,2020-03-18 00:00:00,0,0
48,West Virginia,WV,54.0,2020-03-23 00:00:00,0,0,0,0,2020-05-15 00:00:00
49,Wisconsin,WI,55.0,2020-03-27 00:00:00,2020-03-27 00:00:00,0,2020-03-22 00:00:00,0,2020-05-26 00:00:00
50,Wyoming,WY,56.0,0,0,2020-05-20 00:00:00,0,0,0


In [41]:
Housing.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Stop Initiation of Evictions overall or due to COVID related issues',
       'Stop enforcement of evictions overall or due to COVID related issues',
       'Renter grace period or use of security deposit to pay rent',
       'Order freezing utility shut offs', 'Froze mortgage payments',
       'Lift Eviction Moratorium'],
      dtype='object')

In [42]:
Housing_dates=['Stop Initiation of Evictions overall or due to COVID related issues',
       'Stop enforcement of evictions overall or due to COVID related issues',
       'Renter grace period or use of security deposit to pay rent',
       'Order freezing utility shut offs', 'Froze mortgage payments',
       'Lift Eviction Moratorium']
Housing_dates_updated=change_date(Housing, Housing_dates)
Housing_dates_updated.tail()

,State,State Abbreviation,State FIPS Code,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,State of emergency
46,Virginia,VA,51.0,0.0,88.0,0.0,4.0,0.0,108.0,2020-03-12
47,Washington,WA,53.0,18.0,18.0,0.0,18.0,0.0,0.0,2020-02-29
48,West Virginia,WV,54.0,7.0,0.0,0.0,0.0,0.0,60.0,2020-03-16
49,Wisconsin,WI,55.0,15.0,15.0,0.0,10.0,0.0,75.0,2020-03-12
50,Wyoming,WY,56.0,0.0,0.0,68.0,0.0,0.0,0.0,2020-03-13


#### Unemployment Insurance

In [43]:
Unemployment_Insurance=xls_file.parse('Unemployment Insurance')
Unemployment_Insurance.dropna(inplace=True)
Unemployment_Insurance.shape
Unemployment_Insurance.tail()

(51, 14)

,State,State Abbreviation,State FIPS Code,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks)
46,Virginia,VA,51.0,0,2020-03-17 00:00:00,1,1,0,1,0,378.0,978.0,26.0,39.0
47,Washington,WA,53.0,0,2020-03-18 00:00:00,1,1,1,0,0,790.0,1390.0,26.0,39.0
48,West Virginia,WV,54.0,0,2020-03-19 00:00:00,1,1,0,0,0,424.0,1024.0,26.0,39.0
49,Wisconsin,WI,55.0,0,0,1,1,0,1,0,370.0,970.0,26.0,39.0
50,Wyoming,WY,56.0,1,0,1,0,0,0,0,508.0,1108.0,26.0,39.0


In [44]:
Unemployment_Insurance_dates= ['Waived one week waiting period for unemployment insurance']
Unemployment_Insurance_dates_updated= change_date(Unemployment_Insurance,Unemployment_Insurance_dates)

#### Healthcare Delivery

In [45]:
Healthcare_Delivery=xls_file.parse('Healthcare Delivery')
Healthcare_Delivery.dropna(inplace=True)
Healthcare_Delivery.shape
Healthcare_Delivery.tail()

(51, 12)

,State,State Abbreviation,State FIPS Code,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access
46,Virginia,VA,51.0,2020-03-23 00:00:00,0,0,2020-03-19 00:00:00,2020-03-19 00:00:00,0,0,2020-03-25 00:00:00,0
47,Washington,WA,53.0,2020-03-19 00:00:00,2020-03-10 00:00:00,0,2020-03-25 00:00:00,2020-03-25 00:00:00,1,0,2020-03-19 00:00:00,0
48,West Virginia,WV,54.0,2020-03-30 00:00:00,0,0,2020-03-16 00:00:00,2020-03-17 00:00:00,0,0,2020-04-01 00:00:00,1
49,Wisconsin,WI,55.0,2020-04-20 00:00:00,0,0,2020-03-17 00:00:00,2020-03-12 00:00:00,1,0,0,0
50,Wyoming,WY,56.0,2020-03-27 00:00:00,0,0,0,0,0,0,0,0


In [46]:
Healthcare_Delivery.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Modify Medicaid requirements with 1135 waivers (date of CMS approval)',
       'Reopened ACA enrollment using a special enrollment period',
       'State previously allowed audio-only telehealth',
       'Allow audio-only telehealth',
       'Allow/expand Medicaid telehealth coverage',
       'State had CHIP premium non-payment lock-out period as of January 2019',
       'Supend CHIP premium non-payment lock-outs',
       'Suspended elective medical/dental procedures',
       'Made Effort to Limit Abortion Access'],
      dtype='object')

In [47]:
Healthcare_Delivery_dates=['Modify Medicaid requirements with 1135 waivers (date of CMS approval)',
       'Reopened ACA enrollment using a special enrollment period',
       'Allow audio-only telehealth',
       'Allow/expand Medicaid telehealth coverage',
       'Suspended elective medical/dental procedures']
Healthcare_Delivery_dates_updated=change_date(Healthcare_Delivery,Healthcare_Delivery_dates)

#### Racial_Disparities

In [48]:
Racial_Disparities=xls_file.parse('Racial Disparities')
Racial_Disparities.shape
Racial_Disparities.dropna(inplace=True)
Racial_Disparities.tail()

(53, 28)

,State,State Abbreviation,State FIPS Code,Report testing by race/ethnicity,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Report cases by race/ethnicity,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Report hospitalizations by race/ethnicity,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Report deaths by race/ethnicity,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Report COVID testing for American Indian/Alaska Native individuals (July),Report positive COVID tests for American Indian/Alaska Native individuals (July),Report COVID hospitalizations for American Indian/Alaska Native individuals (July),Report COVID deaths for American Indian/Alaska Native individuals (July),State has at least one American Indian/Alaskan Native reservation (July)
47,Virginia,VA,51.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1.0,1.0,0.0,1.0,1.0
48,Washington,WA,53.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0,1.0,1.0,1.0,1.0
49,West Virginia,WV,54.0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0
50,Wisconsin,WI,55.0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1.0,1.0,0.0,1.0,1.0
51,Wyoming,WY,56.0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0.0,1.0,0.0,1.0,1.0


In [49]:
Racial_Disparities['Report testing by race/ethnicity'].value_counts()

1    41
0    10
Name: Report testing by race/ethnicity, dtype: int64

#### Incarcerated Individuals

In [50]:
Incarcerated_Individuals=xls_file.parse('Incarcerated Individuals')
Incarcerated_Individuals.shape
Incarcerated_Individuals.dropna(inplace=True)
Incarcerated_Individuals.tail()

(52, 9)

,State,State Abbreviation,State FIPS Code,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals
46,Virginia,VA,51.0,2020-03-13 00:00:00,2020-03-13 00:00:00,1,0,0,0
47,Washington,WA,53.0,2020-03-13 00:00:00,0,0,1,0,0
48,West Virginia,WV,54.0,2020-03-11 00:00:00,0,0,0,1,0
49,Wisconsin,WI,55.0,2020-03-13 00:00:00,0,0,1,0,0
50,Wyoming,WY,56.0,2020-03-18 00:00:00,0,1,0,0,0


In [51]:
Incarcerated_Individuals.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Stopped personal visitation in state prisons',
       'Stopped in-person attorney visits in state prisons',
       'Does not charge copays for incarcerated individuals',
       'Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals',
       'Waived all copays during pandemic for incarcerated individuals',
       'Did not waive copays for incarcerated individuals'],
      dtype='object')

In [52]:
Incarcerated_Individuals_dates=['Stopped personal visitation in state prisons',
       'Stopped in-person attorney visits in state prisons']
Incarcerated_Individuals_dates_updated =change_date(Incarcerated_Individuals,Incarcerated_Individuals_dates)
Incarcerated_Individuals_dates_updated.tail()

,State,State Abbreviation,State FIPS Code,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,State of emergency
46,Virginia,VA,51.0,1,1.0,1,0,0,0,2020-03-12
47,Washington,WA,53.0,13,0.0,0,1,0,0,2020-02-29
48,West Virginia,WV,54.0,-5,0.0,0,0,1,0,2020-03-16
49,Wisconsin,WI,55.0,1,0.0,0,1,0,0,2020-03-12
50,Wyoming,WY,56.0,5,0.0,1,0,0,0,2020-03-13


#### Substance Use Disorder Policies

In [53]:
Substance_Use_Disorder_Policies=xls_file.parse('Substance Use Disorder Policies')
Substance_Use_Disorder_Policies.dropna(inplace=True)
Substance_Use_Disorder_Policies.shape
Substance_Use_Disorder_Policies.tail()

(51, 9)

,State,State Abbreviation,State FIPS Code,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medication by opioid treatment programs,Use of telemedicine for schedule II-V prescriptions,Exceptions to emergency oral prescriptions,Waive requirement to obtain separate DEA registration to dispense outside home state
46,Virginia,VA,51.0,0,0,0,0,0,0
47,Washington,WA,53.0,0,0,0,0,0,0
48,West Virginia,WV,54.0,0,0,0,2020-03-31 00:00:00,2020-03-31 00:00:00,0
49,Wisconsin,WI,55.0,0,0,0,0,0,0
50,Wyoming,WY,56.0,0,0,0,0,0,0


In [54]:
Substance_Use_Disorder_Policies.columns

Index(['State', 'State Abbreviation', 'State FIPS Code',
       'Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing',
       'Patients can receive 14-28 take-home doses of opioid medication',
       'Home delivery of take-home medication by opioid treatment programs',
       'Use of telemedicine for schedule II-V prescriptions',
       'Exceptions to emergency oral prescriptions',
       'Waive requirement to obtain separate DEA registration to dispense outside home state'],
      dtype='object')

In [55]:
Substance_Use_Disorder_Policies_dates =['Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing',
       'Patients can receive 14-28 take-home doses of opioid medication',
       'Home delivery of take-home medication by opioid treatment programs',
       'Use of telemedicine for schedule II-V prescriptions',
       'Exceptions to emergency oral prescriptions',
       'Waive requirement to obtain separate DEA registration to dispense outside home state']
Substance_Use_Disorder_Policies_dates_updated=change_date(Substance_Use_Disorder_Policies,Substance_Use_Disorder_Policies_dates)
Substance_Use_Disorder_Policies_dates_updated.head()

,State,State Abbreviation,State FIPS Code,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medication by opioid treatment programs,Use of telemedicine for schedule II-V prescriptions,Exceptions to emergency oral prescriptions,Waive requirement to obtain separate DEA registration to dispense outside home state,State of emergency
0,Alabama,AL,1.0,0.0,0.0,0.0,7.0,0.0,0.0,2020-03-13
1,Alaska,AK,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-11
2,Arizona,AZ,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-11
3,Arkansas,AR,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-11
4,California,CA,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-04


#### State_Characteristics

In [56]:
State_Characteristics=xls_file.parse('State Characteristics')
State_Characteristics.dropna(inplace=True)
State_Characteristics.shape
State_Characteristics.tail()

(51, 11)

,State,State Abbreviation,State FIPS Code,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018
46,Virginia,VA,51,199.13,8517685,42775.0,5783,4.2,10.7,35.9,69359
47,Washington,WA,53,105.69,7535591,71298.0,21577,4.3,10.3,35.1,56877
48,West Virginia,WV,54,74.53,1805832,24230.0,1397,5.8,17.8,49.3,23478
49,Wisconsin,WI,55,88.76,5813568,65496.0,4538,3.2,11.0,36.5,53684
50,Wyoming,WY,56,5.91,577737,97813.0,548,3.9,11.1,36.4,5070


In [57]:
NotesDetails=xls_file.parse('NotesDetails')
NotesDetails.dropna(inplace=True)
NotesDetails.shape
NotesDetails.tail()

(0, 102)

,Unnamed: 0,State of emergency,Date closed K-12 schools,Closed day cares,Reopened Childcare,Date banned visitors to nursing homes,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,Religious Gatherings Exempt Without Clear Social Distance Mandate*,Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Closed restaurants except take out,Reopen restaurants,Closed gyms,Reopened gyms,Closed movie theaters,Reopened movie theaters,Closed Bars,Reopened hair salons/barber shops,Non-essential construction,Reopen Religious Gatherings,Reopen non-essential retail,Reopen bars,Begin to reclose bars,Reclose bars (statewide),Relcose movie theaters (statewide),Reclose gyms (statewide),Reclose indoor dining (statewide),Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Report testing by race/ethnicity,Report cases by race/ethnicity,Report hospitalizations by race/ethnicity,Report deaths by race/ethnicity,Report COVID testing for Native Amerian/Alaska Native individuals,Report positive COVID tests for Native Amerian/Alaska Native individuals,Report COVID hospitalizations for Native Amerian/Alaska Native individuals,Report COVID deaths for Native Amerian/Alaska Native individuals,State has at least one American Indian/Alaskan Native reservation,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Suspended elective medical/dental procedures,Resumed elective medical procedures,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),Made Effort to Limit Abortion Access,Efforts to limit abortion access details,Use of telemedicine/telephone eva

# MERGING ALL POLICY DATA

In [58]:
sheet_names

['Information',
 'State of Emergency',
 'Physical Distance Closures',
 'Stay at Home',
 'Reopening',
 '2nd Closures',
 'Face Masks',
 'Quarantine Rules',
 'Alcohol and Firearms',
 'Housing',
 'Unemployment Insurance',
 'Food Security',
 'Healthcare Delivery',
 'Racial Disparities',
 'Incarcerated Individuals',
 'Substance Use Disorder Policies',
 'Pre-Covid Policies',
 'State Characteristics',
 'NotesDetails']

In [59]:
# To creat a list of the dataframe names I created using the sheet names
sheets=[]
for sheet in sheet_names:
    new_sheet= sheet.replace(' ','_')
    sheets.append(new_sheet)
sheets

['Information',
 'State_of_Emergency',
 'Physical_Distance_Closures',
 'Stay_at_Home',
 'Reopening',
 '2nd_Closures',
 'Face_Masks',
 'Quarantine_Rules',
 'Alcohol_and_Firearms',
 'Housing',
 'Unemployment_Insurance',
 'Food_Security',
 'Healthcare_Delivery',
 'Racial_Disparities',
 'Incarcerated_Individuals',
 'Substance_Use_Disorder_Policies',
 'Pre-Covid_Policies',
 'State_Characteristics',
 'NotesDetails']

In [60]:
data_frames_to_merge= [Physical_Distance_Closures_dates_updated,
 Stay_at_Home_dates_updated,
 Reopening_dates_updated,Second_Closures_dates_updated,
 Face_Masks_dates_updated,
 Quarantine_Rules_dates_updated,
 Alcohol_and_Firearms_dates_updated,
 Housing_dates_updated,Unemployment_Insurance_dates_updated,
 Food_Security_dates_updated,
 Healthcare_Delivery_dates_updated,
 Incarcerated_Individuals_dates_updated,
 Substance_Use_Disorder_Policies_dates_updated]

In [61]:
# This data frames are a yes or no answer only & they don't have dates for the policies .
data_no_dates=[Pre_Covid_Policies,State_Characteristics]

In [62]:
master_date_col=[Physical_Distance_Closures_dates,
 Stay_at_Home_dates,
 Reopening_dates,Second_Closures_dates,
 Face_Masks_dates,
 Quarantine_Rules_dates,
 Alcohol_and_Firearms_dates,
 Housing,Unemployment_Insurance_dates,
 Food_Security_dates,
 Healthcare_Delivery_dates,
 Incarcerated_Individuals_dates,
 Substance_Use_Disorder_Policies_dates]

In [63]:
from functools import reduce
# merge dataframes with dates/State of Emergency column
policies_with_date = reduce(lambda  left,right: pd.merge(left,right,on=['State','State Abbreviation','State FIPS Code','State of emergency'],
                                            how='outer'), data_frames_to_merge)

In [64]:
data_frames_no_dates=[Pre_Covid_Policies,State_Characteristics,policies_with_date]

In [65]:
#merge dataframes with out date columns
policies_df = reduce(lambda  left,right: pd.merge(left,right,on=['State','State Abbreviation','State FIPS Code'],
                                            how='outer'), data_frames_no_dates)

In [66]:
policies_df.head()
policies_df.shape

,State,State Abbreviation,State FIPS Code,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,State of emergency,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medication

(51, 95)

In [67]:
policies_df.to_csv('./covid19_policies_final.csv', index = False)

# CASES and DEATHS by Race

In [68]:
race_df= pd.read_csv('covid 19 Race Data .csv')
race_df.fillna(value=0, inplace =True)
race_df.head()
race_df.shape

,Date,State,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown
0,20200722,AK,2132,750,49.0,0.0,67.0,282.0,76.0,61.0,38.0,809.0,117.0,1045.0,970.0,19.0,11.0,0.0,0.0,2.0,4.0,1.0,0.0,0.0,1.0,0.0,18.0,1.0
1,20200722,AL,71813,22694,20884.0,0.0,267.0,0.0,0.0,0.0,3957.0,24002.0,5457.0,36516.0,29832.0,1364.0,672.0,561.0,0.0,4.0,0.0,0.0,0.0,27.0,100.0,45.0,1126.0,193.0
2,20200722,AR,35246,17406,7521.0,0.0,498.0,101.0,1985.0,0.0,4638.0,3097.0,8411.0,26835.0,0.0,380.0,214.0,100.0,0.0,6.0,0.0,30.0,0.0,27.0,3.0,41.0,336.0,3.0
3,20200722,AS,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200722,AZ,150609,24130,3246.0,34645.0,1096.0,9085.0,0.0,0.0,3034.0,75373.0,34645.0,40591.0,75373.0,2974.0,1128.0,86.0,786.0,32.0,394.0,0.0,0.0,53.0,495.0,786.0,1693.0,495.0


(1680, 28)

In [69]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 28 columns):
Date                            1680 non-null int64
State                           1680 non-null object
Cases_Total                     1680 non-null object
Cases_White                     1680 non-null object
Cases_Black                     1680 non-null float64
Cases_LatinX                    1680 non-null float64
Cases_Asian                     1680 non-null float64
Cases_AIAN                      1680 non-null float64
Cases_NHPI                      1680 non-null float64
Cases_Multiracial               1680 non-null float64
Cases_Other                     1680 non-null float64
Cases_Unknown                   1680 non-null float64
Cases_Ethnicity_Hispanic        1680 non-null float64
Cases_Ethnicity_NonHispanic     1680 non-null float64
Cases_Ethnicity_Unknown         1680 non-null float64
Deaths_Total                    1680 non-null float64
Deaths_White                    16

The data types for columns Cases_White and Cases_Total are object so we have to find the string values in those columns

In [70]:
df_comma_w= race_df[race_df['Cases_White'].str.isdigit() == False]
df_comma_w

,Date,State,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown
1174,20200513,WV,"1,404","1,148",105.0,0.0,0.0,0.0,0.0,0.0,152.0,0.0,0.0,0.0,0.0,59.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0


In [71]:
df_comma_t= race_df[race_df['Cases_Total'].str.isdigit() == False]
df_comma_t

,Date,State,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown
1174,20200513,WV,"1,404","1,148",105.0,0.0,0.0,0.0,0.0,0.0,152.0,0.0,0.0,0.0,0.0,59.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0


need to remove the commas in those values

In [72]:
race_df['Cases_White']=race_df['Cases_White'].str.replace(',', '').astype(float)
race_df['Cases_Total']=race_df['Cases_Total'].str.replace(',', '').astype(float)

In [73]:
race_df.fillna(0,inplace= True)

In [74]:
race_df.isna().sum()

Date                            0
State                           0
Cases_Total                     0
Cases_White                     0
Cases_Black                     0
Cases_LatinX                    0
Cases_Asian                     0
Cases_AIAN                      0
Cases_NHPI                      0
Cases_Multiracial               0
Cases_Other                     0
Cases_Unknown                   0
Cases_Ethnicity_Hispanic        0
Cases_Ethnicity_NonHispanic     0
Cases_Ethnicity_Unknown         0
Deaths_Total                    0
Deaths_White                    0
Deaths_Black                    0
Deaths_LatinX                   0
Deaths_Asian                    0
Deaths_AIAN                     0
Deaths_NHPI                     0
Deaths_Multiracial              0
Deaths_Other                    0
Deaths_Unknown                  0
Deaths_Ethnicity_Hispanic       0
Deaths_Ethnicity_NonHispanic    0
Deaths_Ethnicity_Unknown        0
dtype: int64

In [75]:
race_df.head()

,Date,State,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown
0,20200722,AK,2132.0,750.0,49.0,0.0,67.0,282.0,76.0,61.0,38.0,809.0,117.0,1045.0,970.0,19.0,11.0,0.0,0.0,2.0,4.0,1.0,0.0,0.0,1.0,0.0,18.0,1.0
1,20200722,AL,71813.0,22694.0,20884.0,0.0,267.0,0.0,0.0,0.0,3957.0,24002.0,5457.0,36516.0,29832.0,1364.0,672.0,561.0,0.0,4.0,0.0,0.0,0.0,27.0,100.0,45.0,1126.0,193.0
2,20200722,AR,35246.0,17406.0,7521.0,0.0,498.0,101.0,1985.0,0.0,4638.0,3097.0,8411.0,26835.0,0.0,380.0,214.0,100.0,0.0,6.0,0.0,30.0,0.0,27.0,3.0,41.0,336.0,3.0
3,20200722,AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200722,AZ,150609.0,24130.0,3246.0,34645.0,1096.0,9085.0,0.0,0.0,3034.0,75373.0,34645.0,40591.0,75373.0,2974.0,1128.0,86.0,786.0,32.0,394.0,0.0,0.0,53.0,495.0,786.0,1693.0,495.0


In [76]:
race_by_state = race_df.groupby(by='State', as_index=False).sum()
race_by_state.shape
race_by_state.tail(10) 

(56, 28)

,State,Date,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown
46,TN,606017080,905690.0,379914.0,187365.0,0.0,12985.0,641.0,420.0,0.0,126128.0,198237.0,190282.0,459282.0,256056.0,12390.0,7036.0,4234.0,0.0,240.0,0.0,10.0,0.0,627.0,243.0,931.0,11049.0,410.0
47,TX,606017080,3098412.0,130301.0,61511.0,176174.0,13939.0,0.0,0.0,0.0,2734.0,2713753.0,174441.0,208009.0,2712395.0,52853.0,6160.0,2004.0,3958.0,262.0,0.0,0.0,0.0,62.0,39626.0,3928.0,8623.0,40336.0
48,UT,606017080,405160.0,149833.0,11192.0,125075.0,9339.0,10613.0,15148.0,0.0,48392.0,44326.0,164500.0,194173.0,45797.0,3278.0,1216.0,60.0,548.0,111.0,148.0,130.0,0.0,179.0,1093.0,548.0,1637.0,100.0
49,VA,606017080,1270048.0,330537.0,191624.0,243309.0,23833.0,869.0,0.0,2355.0,129675.0,347846.0,392109.0,478980.0,340695.0,36137.0,19729.0,7947.0,2184.0,1158.0,20.0,0.0,2.0,2076.0,3021.0,3460.0,27728.0,3988.0
50,VI,606017080,2485.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2485.0,0.0,0.0,0.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131.0,0.0,0.0,0.0
51,VT,606017080,28984.0,24815.0,1926.0,51.0,822.0,41.0,0.0,0.0,459.0,921.0,827.0,26057.0,2102.0,1449.0,1346.0,0.0,0.0,54.0,0.0,0.0,0.0,47.0,2.0,2.0,1432.0,15.0
52,WA,606017080,737119.0,198259.0,30892.0,205091.0,34336.0,7918.0,13346.0,8831.0,11257.0,227189.0,201053.0,294017.0,217765.0,32511.0,21409.0,985.0,3268.0,2774.0,405.0,255.0,397.0,656.0,2362.0,3182.0,25791.0,2212.0
53,WI,606017080,602924.0,334811.0,108750.0,0.0,21659.0,5956.0,0.0,0.0,66348.0,65400.0,176333.0,353664.0,72926.0,16865.0,11407.0,4355.0,0.0,438.0,215.0,0.0,0.0,173.0,277.0,1691.0,14842.0,332.0
54,WV,606017080,65403.0,52046.0,4663.0,0.0,0.0,0.0,0.0,0.0,8776.0,25.0,0.0,0.0,0.0,1842.0,1161.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,619.0,0.0,0.0,0.0
55,WY,606017080,25298.0,12783.0,276.0,3185.0,155.0,6273.0,57.0,0.0,895.0,3312.0,3263.0,20173.0,3299.0,376.0,123.0,0.0,18.0,0.0,127.0,0.0,0.0,0.0,108.0,18.0,255.0,16.0


In [77]:
race_by_state['Cases_White_percent']= round(race_by_state['Cases_White']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_Black_percent']= round(race_by_state['Cases_Black']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_LatinX_percent']= round(race_by_state['Cases_LatinX']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_Asian_percent']= round(race_by_state['Cases_Asian']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_AIAN_percent']= round(race_by_state['Cases_AIAN']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_NHPI_percent']= round(race_by_state['Cases_NHPI']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_Multiracial_percent']= round(race_by_state['Cases_Multiracial']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_Other_percent']= round(race_by_state['Cases_Other']/race_by_state['Cases_Total']*100,2)
race_by_state['Cases_Unknown_percent']= round(race_by_state['Cases_Unknown']/race_by_state['Cases_Total']*100,2)
race_by_state.head()

,State,Date,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown,Cases_White_percent,Cases_Black_percent,Cases_LatinX_percent,Cases_Asian_percent,Cases_AIAN_percent,Cases_NHPI_percent,Cases_Multiracial_percent,Cases_Other_percent,Cases_Unknown_percent
0,AK,606017080,21110.0,10675.0,508.0,0.0,1480.0,2243.0,609.0,661.0,459.0,4475.0,1432.0,14178.0,5703.0,317.0,147.0,0.0,0.0,42.0,50.0,21.0,0.0,0.0,27.0,0.0,255.0,62.0,50.57,2.41,0.00,7.01,10.63,2.88,3.13,2.17,21.20
1,AL,606017080,731345.0,259537.0,262248.0,0.0,3323.0,0.0,0.0,0.0,37643.0,168536.0,63683.0,450709.0,216933.0,19350.0,9492.0,8497.0,0.0,105.0,0.0,0.0,0.0,289.0,946.0,510.0,16484.0,2341.0,35.49,35.86,0.00,0.45,0.00,0.00,0.00,5.15,23.04
2,AR,606017080,361431.0,173073.0,89306.0,232.0,4395.0,981.0,19541.0,154.0,41638.0,32059.0,74794.0,268920.0,9916.0,4921.0,2728.0,1418.0,0.0,43.0,0.0,292.0,0.0,285.0,92.0,326.0,3948.0,300.0,47.89,24.71,0.06,1.22,0.27,5.41,0.04,11.52,8.87
3,AS,606017080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AZ,606017080,1336330.0,231713.0,32411.0,309291.0,10693.0,108902.0,128.0,0.0,27289.0,615903.0,308639.0,408866.0,611324.0,32963.0,14009.0,973.0,6861.0,375.0,5419.0,2.0,0.0,592.0,4732.0,6838.0,21228.0,4641.0,17.34,2.43,23.14,0.80,8.15,0.01,0.00,2.04,46.09


In [78]:
race_by_state['Deaths_White_percent']= round(race_by_state['Deaths_White']/race_by_state['Deaths_Total']*100,2)
race_by_state['Deaths_Black_percent']= round(race_by_state['Deaths_Black']/race_by_state['Deaths_Total']*100,2)
race_by_state['DeathsDeaths_LatinX_percent']= round(race_by_state['Deaths_LatinX']/race_by_state['Deaths_Total']*100,2)
race_by_state['Deaths_Asian_percent']= round(race_by_state['Deaths_Asian']/race_by_state['Deaths_Total']*100,2)
race_by_state['Deaths_AIAN_percent']= round(race_by_state['Deaths_AIAN']/race_by_state['Deaths_Total']*100,2)
race_by_state['Deaths_NHPI_percent']= round(race_by_state['Deaths_NHPI']/race_by_state['Deaths_Total']*100,2)
race_by_state['Deaths_Multiracial_percent']= round(race_by_state['Deaths_Multiracial']/race_by_state['Deaths_Total']*100,2)
race_by_state['Deaths_Other_percent']= round(race_by_state['Deaths_Other']/race_by_state['Deaths_Total']*100,2)
race_by_state['Deaths_Unknown_percent']= round(race_by_state['Deaths_Unknown']/race_by_state['Deaths_Total']*100,2)
race_by_state.head()

,State,Date,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown,Cases_White_percent,Cases_Black_percent,Cases_LatinX_percent,Cases_Asian_percent,Cases_AIAN_percent,Cases_NHPI_percent,Cases_Multiracial_percent,Cases_Other_percent,Cases_Unknown_percent,Deaths_White_percent,Deaths_Black_percent,DeathsDeaths_LatinX_percent,Deaths_Asian_percent,Deaths_AIAN_percent,Deaths_NHPI_percent,Deaths_Multiracial_percent,Deaths_Other_percent,Deaths_Unknown_percent
0,AK,606017080,21110.0,10675.0,508.0,0.0,1480.0,2243.0,609.0,661.0,459.0,4475.0,1432.0,14178.0,5703.0,317.0,147.0,0.0,0.0,42.0,50.0,21.0,0.0,0.0,27.0,0.0,255.0,62.0,50.57,2.41,0.00,7.01,10.63,2.88,3.13,2.17,21.20,46.37,0.00,0.00,13.25,15.77,6.62,0.0,0.00,8.52
1,AL,606017080,731345.0,259537.0,262248.0,0.0,3323.0,0.0,0.0,0.0,37643.0,168536.0,63683.0,450709.0,216933.0,19350.0,9492.0,8497.0,0.0,105.0,0.0,0.0,0.0,289.0,946.0,510.0,16484.0,2341.0,35.49,35.86,0.00,0.45,0.00,0.00,0.00,5.15,23.04,49.05,43.91,0.00,0.54,0.00,0.00,0.0,1.49,4.89
2,AR,606017080,361431.0,173073.0,89306.0,232.0,4395.0,981.0,19541.0,154.0,41638.0,32059.0,74794.0,268920.0,9916.0,4921.0,2728.0,1418.0,0.0,43.0,0.0,292.0,0.0,285.0,92.0,326.0,3948.0,300.0,47.89,24.71,0.06,1.22,0.27,5.41,0.04,11.52,8.87,55.44,28.82,0.00,0.87,0.00,5.93,0.0,5.79,1.87
3,AS,606017080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AZ,606017080,1336330.0,231713.0,32411.0,309291.0,10693.0,108902.0,128.0,0.0,27289.0,615903.0,308639.0,408866.0,611324.0,32963.0,14009.0,973.0,6861.0,375.0,5419.0,2.0,0.0,592.0,4732.0,6838.0,21228.0,4641.0,17.34,2.43,23.14,0.80,8.15,0.01,0.00,2.04,46.09,42.50,2.95,20.81,1.14,16.44,0.01,0.0,1.80,14.36


In [79]:
race_by_state.fillna(0, inplace =True)

In [80]:
race_by_state.columns

Index(['State', 'Date', 'Cases_Total', 'Cases_White', 'Cases_Black',
       'Cases_LatinX', 'Cases_Asian', 'Cases_AIAN', 'Cases_NHPI',
       'Cases_Multiracial', 'Cases_Other', 'Cases_Unknown',
       'Cases_Ethnicity_Hispanic', 'Cases_Ethnicity_NonHispanic',
       'Cases_Ethnicity_Unknown', 'Deaths_Total', 'Deaths_White',
       'Deaths_Black', 'Deaths_LatinX', 'Deaths_Asian', 'Deaths_AIAN',
       'Deaths_NHPI', 'Deaths_Multiracial', 'Deaths_Other', 'Deaths_Unknown',
       'Deaths_Ethnicity_Hispanic', 'Deaths_Ethnicity_NonHispanic',
       'Deaths_Ethnicity_Unknown', 'Cases_White_percent',
       'Cases_Black_percent', 'Cases_LatinX_percent', 'Cases_Asian_percent',
       'Cases_AIAN_percent', 'Cases_NHPI_percent', 'Cases_Multiracial_percent',
       'Cases_Other_percent', 'Cases_Unknown_percent', 'Deaths_White_percent',
       'Deaths_Black_percent', 'DeathsDeaths_LatinX_percent',
       'Deaths_Asian_percent', 'Deaths_AIAN_percent', 'Deaths_NHPI_percent',
       'Deaths_Multirac

Selecting unly the necessary columns

In [81]:
race_case_death_percent = race_by_state[[ 'State', 'Cases_Total', 'Deaths_Total','Cases_White_percent',
       'Cases_Black_percent', 'Cases_LatinX_percent', 'Cases_Asian_percent',
       'Cases_AIAN_percent', 'Cases_NHPI_percent', 'Cases_Multiracial_percent',
       'Cases_Other_percent', 'Cases_Unknown_percent', 'Deaths_White_percent',
       'Deaths_Black_percent', 'DeathsDeaths_LatinX_percent',
       'Deaths_Asian_percent', 'Deaths_AIAN_percent', 'Deaths_NHPI_percent',
       'Deaths_Multiracial_percent', 'Deaths_Other_percent',
       'Deaths_Unknown_percent']].copy()

## Merge the policies data with race data

The race data set has 56 states but the policies data has 51 state. We need to remove the extra states in the race data set since we don't have the info by race for those states.

In [82]:
different_states= list(set(race_case_death_percent['State'])- set(policies_df['State Abbreviation']))
different_states

['MP', 'AS', 'GU', 'PR', 'VI']

In [83]:
race_case_death_percent_clean= race_case_death_percent[~race_case_death_percent['State'].isin(different_states)]
race_case_death_percent_clean.head()
race_case_death_percent_clean.shape

,State,Cases_Total,Deaths_Total,Cases_White_percent,Cases_Black_percent,Cases_LatinX_percent,Cases_Asian_percent,Cases_AIAN_percent,Cases_NHPI_percent,Cases_Multiracial_percent,Cases_Other_percent,Cases_Unknown_percent,Deaths_White_percent,Deaths_Black_percent,DeathsDeaths_LatinX_percent,Deaths_Asian_percent,Deaths_AIAN_percent,Deaths_NHPI_percent,Deaths_Multiracial_percent,Deaths_Other_percent,Deaths_Unknown_percent
0,AK,21110.0,317.0,50.57,2.41,0.00,7.01,10.63,2.88,3.13,2.17,21.20,46.37,0.00,0.00,13.25,15.77,6.62,0.00,0.00,8.52
1,AL,731345.0,19350.0,35.49,35.86,0.00,0.45,0.00,0.00,0.00,5.15,23.04,49.05,43.91,0.00,0.54,0.00,0.00,0.00,1.49,4.89
2,AR,361431.0,4921.0,47.89,24.71,0.06,1.22,0.27,5.41,0.04,11.52,8.87,55.44,28.82,0.00,0.87,0.00,5.93,0.00,5.79,1.87
4,AZ,1336330.0,32963.0,17.34,2.43,23.14,0.80,8.15,0.01,0.00,2.04,46.09,42.50,2.95,20.81,1.14,16.44,0.01,0.00,1.80,14.36
5,CA,4427497.0,125500.0,12.71,3.20,36.42,5.05,0.14,0.46,0.51,8.41,33.07,31.62,9.21,38.89,14.04,0.33,0.56,0.56,1.09,3.69


(51, 21)

In [85]:
policies_race = pd.merge(policies_df, race_case_death_percent_clean,left_on='State Abbreviation',
    right_on='State',how='left')
policies_race.shape
policies_race.head()

(51, 116)

,State_x,State Abbreviation,State FIPS Code,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,State of emergency,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medicati

## Check relationships between deaths 

In [86]:
policies_race.head()

,State_x,State Abbreviation,State FIPS Code,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,State of emergency,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medicati

In [87]:
policies_race['Cases_Total_percent'] = round(policies_race['Cases_Total']/policies_race['Population 2018']*100,2)
policies_race['Deaths_Total_percent']= round(policies_race['Deaths_Total']/policies_race['Population 2018']*100,2)
policies_race['homeles_percent']= round(policies_race['Number Homeless (2019)']/policies_race['Population 2018']*100,2)
policies_race['All-cause deaths 2018_percent']= round(policies_race['All-cause deaths 2018']/policies_race['Population 2018']*100,2)
policies_race.head()

,State_x,State Abbreviation,State FIPS Code,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,State of emergency,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medicati

In [88]:
policies_race.head()

,State_x,State Abbreviation,State FIPS Code,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,State of emergency,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medicati

In [95]:
# Non numeric column Lists to drop before scaling 
drop_list =['State_x','State Abbreviation','Population 2018','Square Miles','State of emergency','State FIPS Code','Number Homeless (2019)','State_y','Cases_Total','Deaths_Total']
policies_race_clean = policies_race.drop(drop_list, axis=1 )
policies_race_clean.head()

,Paid sick leave,Medicaid Expansion,Population density per square miles,Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medication by opioid treatment programs,Use of telemedicine for schedule II-V prescriptions,Exceptions to emergency oral p

#### Checking if all values are real numbers before scaling

In [96]:
for col in policies_race_clean.columns:
    if policies_race_clean[col].dtype== 'object':
        print(col)

No legal enforcement of face mask mandate


In [97]:
# I kept getting an error that 'can't convert string to float the following code will find rows with at least one non real value
policies_race_clean[~policies_race_clean.applymap(np.isreal).all(1)] 

,Paid sick leave,Medicaid Expansion,Population density per square miles,Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medication by opioid treatment programs,Use of telemedicine for schedule II-V prescriptions,Exceptions to emergency oral p

There was a value 'f' on the column it suppoese to be 0 0r 1 since the other policies related to this column are 0 we will impute this with 0.

In [98]:
policies_race_clean['No legal enforcement of face mask mandate']=policies_race_clean['No legal enforcement of face mask mandate'].apply(lambda x: float(x) if x!="f" else 0)

In [99]:
policies_race_clean=policies_race_clean.astype(float)

# Census Data

#### Population and Poverty - State

In [100]:
# https://covid19.census.gov/datasets/population-and-poverty-counties/data?geometry=138.304%2C-16.868%2C-137.673%2C72.108
# Population and Poverty - Counties

rawdata_popl_povty = pd.read_csv('./Population_and_Poverty_States.csv')
print(rawdata_popl_povty.shape)
rawdata_popl_povty.head()

(52, 60)


,OBJECTID,GEO_ID,GEO_NAME,Total Households Below the Poverty Level,Total Households Below the Poverty Level *margin of error,Households: Income Below Poverty Level (%),Population for whom Poverty Status is Determined,Population for whom Poverty Status is Determined *margin of error,Income to Poverty Ratio < 0.50,Income to Poverty Ratio < 0.50 *margin of error,Income to Poverty Ratio 0.50-0.99,Income to Poverty Ratio 0.50-0.99 *margin of error,Income to Poverty Ratio 1.00-1.24,Income to Poverty Ratio 1.00-1.24 *margin of error,Income to Poverty Ratio 1.25-1.49,Income to Poverty Ratio 1.25-1.49 *margin of error,Income to Poverty Ratio 1.50-1.84,Income to Poverty Ratio 1.50-1.84 *margin of error,Income to Poverty Ratio 1.85-1.99,Income to Poverty Ratio 1.85-1.99 *margin of error,Income to Poverty Ratio 2.00+,Income to Poverty Ratio 2.00+ *margin of error,All Households Below Poverty Level,All Households Above Poverty Level,Married-Couple Family Households Below Poverty Level,Married-Couple Family Households Below Poverty Level *margin of error,Married-Couple Family Households Above Poverty Level,Married-Couple Family Households Above Poverty Level *margin of error,"Other Families HHlds w/Male Householder, with no wife present, Below Poverty Level","Other Families HHlds w/Male Householder, with no wife present, Below Poverty Level *margin of error","Other Families HHlds w/Male Householder, with no wife present, Above Poverty Level","Other Families HHlds w/Male Householder, with no wife present, Above Poverty Level *margin of error","Other Families Hhlds w/Female Householder, with no husband present, Below Poverty Level","Other Families Hhlds w/Female Householder, with no husband present, Below Poverty Level *margin of error","Other Families Hhlds w/Female Householder, with no husband present, Above Poverty Level","Other Families Hhlds w/Female Householder, with no husband present, Above Poverty Level *margin of error",NonFamily Hhlds w/Male Householder Below Poverty Level,NonFamily Hhlds w/Male Householder Below Poverty Level *margin of error,NonFamily Hhlds w/Male Householder Above Poverty Level,NonFamily Hhlds w/Male Householder Above Poverty Level *margin of error,NonFamily Hhlds w/Female Householder Below Poverty Level,NonFamily Hhlds w/Female Householder Below Poverty Level *margin of error,NonFamily Hhlds w/Female Householder Above Poverty Level,NonFamily Hhlds w/Female Householder Above Poverty Level *margin of error,"Percentage of families whose income in the past 12 months is below the poverty level with female householder, no husband present) - With related children of the householder under 18 years","Percentage of families whose income in the past 12 months is below the poverty level with female householder, no husband present) - With related children of the householder under 18 years *margin of error","Percentage of families whose income in the past 12 months is below the poverty level with female householder, no husband present) - With related children of the householder under 5 years","Percentage of families whose income in the past 12 months is below the poverty level with female householder, no husband present) - With related children of the householder under 5 years *margin of error",Percentage of people whose income in the past 12 months is below the poverty level,Percentage of people whose income in the past 12 months is below the poverty level - All people *margin of error,Percentage of people whose income in the past 12 months is below the poverty level - Related children of the householder under 5 years,Percentage of people whose income in the past 12 months is below the poverty level - Related children of the householder under 5 years *margin of error,Percentage of people whose income in the past 12 months is below the poverty level - 65 years and over,Percentage of people whose income in the past 12 months is below the poverty level - 65 years and over *margin of error,Total Married-Couple Family Households

In [101]:
rawdata_popl_povty.columns

Index([' OBJECTID', ' GEO_ID', ' GEO_NAME',
       ' Total Households Below the Poverty Level',
       ' Total Households Below the Poverty Level *margin of error',
       ' Households: Income Below Poverty Level (%)',
       ' Population for whom Poverty Status is Determined',
       ' Population for whom Poverty Status is Determined *margin of error',
       ' Income to Poverty Ratio < 0.50',
       ' Income to Poverty Ratio < 0.50 *margin of error',
       ' Income to Poverty Ratio 0.50-0.99',
       ' Income to Poverty Ratio 0.50-0.99 *margin of error',
       ' Income to Poverty Ratio 1.00-1.24',
       ' Income to Poverty Ratio 1.00-1.24 *margin of error',
       ' Income to Poverty Ratio 1.25-1.49',
       ' Income to Poverty Ratio 1.25-1.49 *margin of error',
       ' Income to Poverty Ratio 1.50-1.84',
       ' Income to Poverty Ratio 1.50-1.84 *margin of error',
       ' Income to Poverty Ratio 1.85-1.99',
       ' Income to Poverty Ratio 1.85-1.99 *margin of error',
       '

In [102]:
rawdata_popl_povty.columns = [x.lstrip().replace(' ', '') for x in rawdata_popl_povty.columns]
rawdata_popl_povty.columns = [x.replace(',', '') for x in rawdata_popl_povty.columns]
rawdata_popl_povty.columns

Index(['OBJECTID', 'GEO_ID', 'GEO_NAME', 'TotalHouseholdsBelowthePovertyLevel',
       'TotalHouseholdsBelowthePovertyLevel*marginoferror',
       'Households:IncomeBelowPovertyLevel(%)',
       'PopulationforwhomPovertyStatusisDetermined',
       'PopulationforwhomPovertyStatusisDetermined*marginoferror',
       'IncometoPovertyRatio<0.50', 'IncometoPovertyRatio<0.50*marginoferror',
       'IncometoPovertyRatio0.50-0.99',
       'IncometoPovertyRatio0.50-0.99*marginoferror',
       'IncometoPovertyRatio1.00-1.24',
       'IncometoPovertyRatio1.00-1.24*marginoferror',
       'IncometoPovertyRatio1.25-1.49',
       'IncometoPovertyRatio1.25-1.49*marginoferror',
       'IncometoPovertyRatio1.50-1.84',
       'IncometoPovertyRatio1.50-1.84*marginoferror',
       'IncometoPovertyRatio1.85-1.99',
       'IncometoPovertyRatio1.85-1.99*marginoferror',
       'IncometoPovertyRatio2.00+', 'IncometoPovertyRatio2.00+*marginoferror',
       'AllHouseholdsBelowPovertyLevel', 'AllHouseholdsAbovePove

In [103]:
# Add fips code to dataset

rawdata_popl_povty['FIPS_CODE'] = rawdata_popl_povty['GEO_ID'].apply(lambda x: x.replace('0400000US',''))
rawdata_popl_povty['FIPS_CODE'] = rawdata_popl_povty['FIPS_CODE'].astype(int)
rawdata_popl_povty.head()


,OBJECTID,GEO_ID,GEO_NAME,TotalHouseholdsBelowthePovertyLevel,TotalHouseholdsBelowthePovertyLevel*marginoferror,Households:IncomeBelowPovertyLevel(%),PopulationforwhomPovertyStatusisDetermined,PopulationforwhomPovertyStatusisDetermined*marginoferror,IncometoPovertyRatio<0.50,IncometoPovertyRatio<0.50*marginoferror,IncometoPovertyRatio0.50-0.99,IncometoPovertyRatio0.50-0.99*marginoferror,IncometoPovertyRatio1.00-1.24,IncometoPovertyRatio1.00-1.24*marginoferror,IncometoPovertyRatio1.25-1.49,IncometoPovertyRatio1.25-1.49*marginoferror,IncometoPovertyRatio1.50-1.84,IncometoPovertyRatio1.50-1.84*marginoferror,IncometoPovertyRatio1.85-1.99,IncometoPovertyRatio1.85-1.99*marginoferror,IncometoPovertyRatio2.00+,IncometoPovertyRatio2.00+*marginoferror,AllHouseholdsBelowPovertyLevel,AllHouseholdsAbovePovertyLevel,Married-CoupleFamilyHouseholdsBelowPovertyLevel,Married-CoupleFamilyHouseholdsBelowPovertyLevel*marginoferror,Married-CoupleFamilyHouseholdsAbovePovertyLevel,Married-CoupleFamilyHouseholdsAbovePovertyLevel*marginoferror,OtherFamiliesHHldsw/MaleHouseholderwithnowifepresentBelowPovertyLevel,OtherFamiliesHHldsw/MaleHouseholderwithnowifepresentBelowPovertyLevel*marginoferror,OtherFamiliesHHldsw/MaleHouseholderwithnowifepresentAbovePovertyLevel,OtherFamiliesHHldsw/MaleHouseholderwithnowifepresentAbovePovertyLevel*marginoferror,OtherFamiliesHhldsw/FemaleHouseholderwithnohusbandpresentBelowPovertyLevel,OtherFamiliesHhldsw/FemaleHouseholderwithnohusbandpresentBelowPovertyLevel*marginoferror,OtherFamiliesHhldsw/FemaleHouseholderwithnohusbandpresentAbovePovertyLevel,OtherFamiliesHhldsw/FemaleHouseholderwithnohusbandpresentAbovePovertyLevel*marginoferror,NonFamilyHhldsw/MaleHouseholderBelowPovertyLevel,NonFamilyHhldsw/MaleHouseholderBelowPovertyLevel*marginoferror,NonFamilyHhldsw/MaleHouseholderAbovePovertyLevel,NonFamilyHhldsw/MaleHouseholderAbovePovertyLevel*marginoferror,NonFamilyHhldsw/FemaleHouseholderBelowPovertyLevel,NonFamilyHhldsw/FemaleHouseholderBelowPovertyLevel*marginoferror,NonFamilyHhldsw/FemaleHouseholderAbovePovertyLevel,NonFamilyHhldsw/FemaleHouseholderAbovePovertyLevel*marginoferror,Percentageoffamilieswhoseincomeinthepast12monthsisbelowthepovertylevelwithfemalehouseholdernohusbandpresent)-Withrelatedchildrenofthehouseholderunder18years,Percentageoffamilieswhoseincomeinthepast12monthsisbelowthepovertylevelwithfemalehouseholdernohusbandpresent)-Withrelatedchildrenofthehouseholderunder18years*marginoferror,Percentageoffamilieswhoseincomeinthepast12monthsisbelowthepovertylevelwithfemalehouseholdernohusbandpresent)-Withrelatedchildrenofthehouseholderunder5years,Percentageoffamilieswhoseincomeinthepast12monthsisbelowthepovertylevelwithfemalehouseholdernohusbandpresent)-Withrelatedchildrenofthehouseholderunder5years*marginoferror,Percentageofpeoplewhoseincomeinthepast12monthsisbelowthepovertylevel,Percentageofpeoplewhoseincomeinthepast12monthsisbelowthepovertylevel-Allpeople*marginoferror,Percentageofpeoplewhoseincomeinthepast12monthsisbelowthepovertylevel-Relatedchildrenofthehouseholderunder5years,Percentageofpeoplewhoseincomeinthepast12monthsisbelowthepovertylevel-Relatedchildrenofthehouseholderunder5years*marginoferror,Percentageofpeoplewhoseincomeinthepast12monthsisbelowthepovertylevel-65yearsandover,Percentageofpeoplewhoseincomeinthepast12monthsisbelowthepovertylevel-65yearsandover*marginoferror,TotalMarried-CoupleFamilyHouseholds,TotalMarried-CoupleFamilyHouseholds*marginoferror,TotalOtherFamiliesHHldsw/MaleHouseholder,TotalOtherFamiliesHHldsw/MaleHouseholder*marginoferror,TotalOtherFamiliesHhldsw/FemaleHousehiolder,TotalOtherFamiliesHhldsw/FemaleHousehiolder*marginoferror,FIPS_CODE
0,1,0400000US01,Alabama,319903,3830,17.20,4742208,1024,360959,8202,468441,10030,253831,5720,234252,7044,341805,9490,133637,4840,2949283,17184,319903,1540366,50964,1732,832422,6371,13914,926,64609,1968,95188,1946,172808,2921,64975,1617,222247,2960,94862,1973,248280,2751,46.9,0.9,49.5,2.6,17.5,0.3,28.4,0.9,10.3,0.2,883386.0,6445.0,78523.0,2159.

In [104]:
rawdata_popl_povty.isnull().sum()

OBJECTID                                                                                                                                                                      0
GEO_ID                                                                                                                                                                        0
GEO_NAME                                                                                                                                                                      0
TotalHouseholdsBelowthePovertyLevel                                                                                                                                           0
TotalHouseholdsBelowthePovertyLevel*marginoferror                                                                                                                             0
Households:IncomeBelowPovertyLevel(%)                                                                                   

#### Language Spoken at Home - State

In [105]:
# https://covid19.census.gov/datasets/language-spoken-at-home-counties/data?geometry=138.304%2C-16.868%2C-137.673%2C72.108
# Language Spoken at Home - Counties

rawdata_lang_spkn = pd.read_csv('./Language_Spoken_at_Home_States.csv')
rawdata_lang_spkn.columns = [x.lstrip().replace(' ', '') for x in rawdata_lang_spkn.columns]
print(rawdata_lang_spkn.shape)
rawdata_lang_spkn.head()

(52, 122)


,OBJECTID,GEO_ID,GEO_NAME,FIPS_CODE,EnglishOnly-Pop5-17,EnglishOnly-Pop5-17*marginoferror,EnglishOnly-Pop18-64,EnglishOnly-Pop18-64*marginoferror,EnglishOnly-Pop65+,EnglishOnly-Pop65+*marginoferror,"LanguageSpokenatHomebyAge-Spanish-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-Spanish-Speak""lessthanverywell""5andover*marginoferrorEnglish","LanguageSpokenatHomebyAge-OtherIndo-European-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherIndo-European-SpeakEnglish""lessthanverywell""5andover*marginoferror","LanguageSpokenatHomebyAge-AsianPacificIslands-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-AsianPacificIslands-SpeakEnglish""lessthanverywell""5andover*marginoferror","LanguageSpokenatHomebyAge-OtherLanguages-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherLanguages-SpeakEnglish""lessthanverywell""5andover*marginoferror",LanguageSpokenatHome-Spanish(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-OtherIndo-European(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-AsianandPacificIsland(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-Other(LimitedEnglishHHs/allLimitedEnglishHHs),EnglishOnly-Total,EnglishOnly-Total*marginoferror,Spanish-Pop5-17,Spanish-Pop5-17*marginoferror,Spanish-Pop18-64,Spanish-Pop18-64*marginoferror,Spanish-Pop65+,Spanish-Pop65+*marginoferror,Spanish-Total,Spanish-Total*marginoferror,Spanish-SpeakEnglishverywell-Pop5-17,Spanish-SpeakEnglishverywell-Pop5-17*marginoferror,Spanish-SpeakEnglishverywell-Pop18-64,Spanish-SpeakEnglishverywell-Pop18-64*marginoferror,Spanish-SpeakEnglishverywell-Pop65+,Spanish-SpeakEnglishverywell-Pop65+*marginoferror,Spanish-SpeakEnglishverywell-Total,Spanish-SpeakEnglishverywell-Total*marginoferror,Spanish-Englishlessthanverywell-Pop5-17,Spanish-Englishlessthanverywell-Pop5-17*marginoferror,Spanish-Englishlessthanverywell-Pop18-64,Spanish-Englishlessthanverywell-Pop18-64*marginoferror,Spanish-Englishlessthanverywell-Pop65+,Spanish-Englishlessthanverywell-Pop65+*marginoferror,Spanish-Englishlessthanverywell-Total,Spanish-Englishlessthanverywell-Total*marginoferror,OtherIndo-European-Pop5-17,OtherIndo-European-Pop5-17*marginoferror,OtherIndo-European-Pop18-64,OtherIndo-European-Pop18-64*marginoferror,OtherIndo-European-Pop65+,OtherIndo-European-Pop65+*marginoferror,OtherIndo-European-Total,OtherIndo-European-Total*marginoferror,OtherIndo-European-SpeakEnglishverywell-Pop5-17,OtherIndo-European-SpeakEnglishverywell-Pop5-17*marginoferror,OtherIndo-European-SpeakEnglishverywell-Pop18-64,OtherIndo-European-SpeakEnglishverywell-Pop18-64*marginoferror,OtherIndo-European-SpeakEnglishverywell-Pop65+,OtherIndo-European-SpeakEnglishverywell-Pop65+*marginoferror,OtherIndo-European-SpeakEnglishverywell-Total,OtherIndo-European-SpeakEnglishverywell-Total*marginoferror,OtherIndo-European-Englishlessthanverywell-Pop5-17,OtherIndo-European-Englishlessthanverywell-Pop5-17*marginoferror,OtherIndo-European-Englishlessthanverywell-Pop18-64,OtherIndo-European-Englishlessthanverywell-Pop18-64*marginoferror,OtherIndo-European-Englishlessthanverywell-Pop65+,OtherIndo-European-Englishlessthanverywell-Pop65+*marginoferror,OtherIndo-European-Englishlessthanverywell-Total,OtherIndo-European-Englishlessthanverywell-Total*marginoferror,Asian-PacificIsland-Pop5-17,Asian-PacificIsland-Pop5-17*marginoferror,Asian-PacificIsland-Pop18-64,Asian-PacificIsland-Pop18-64*marginoferror,Asian-PacificIsland-Pop65+,Asian-PacificIsland-Pop65+*marginoferror,Asian-PacificIsland-Total,Asian-PacificIsland-Total*marginoferror,Asian-PacificIsland-SpeakEnglishverywell-Pop5-17,Asian-PacificIsland-SpeakEnglishverywell-Pop5-17*marginoferror,Asian-PacificIsland-SpeakEnglishverywell-Pop18-64,Asian-PacificIsland-SpeakEnglishverywell-Pop18-64*marginoferror,Asian-PacificIsland-SpeakEnglishverywell-Pop65+,Asian-PacificIsland-SpeakEnglishverywell-Pop65+*marginoferror,Asian-PacificIsland-SpeakEnglishverywell-Tota

In [106]:
rawdata_lang_spkn.isnull().sum()

OBJECTID                                                                                              0
GEO_ID                                                                                                0
GEO_NAME                                                                                              0
FIPS_CODE                                                                                             0
EnglishOnly-Pop5-17                                                                                   0
EnglishOnly-Pop5-17*marginoferror                                                                     0
EnglishOnly-Pop18-64                                                                                  0
EnglishOnly-Pop18-64*marginoferror                                                                    0
EnglishOnly-Pop65+                                                                                    0
EnglishOnly-Pop65+*marginoferror                                

#### Computer and Internet Use - State

In [107]:
# https://covid19.census.gov/datasets/computer-and-internet-use-counties/data?geometry=138.304%2C-16.868%2C-137.673%2C72.108
# Computer and Internet Use - Counties

rawdata_comptr_internt = pd.read_csv('./Computer_and_Internet_Use_States.csv')
print(rawdata_comptr_internt.shape)
rawdata_comptr_internt.columns = [x.lstrip().replace(' ', '') for x in rawdata_comptr_internt.columns]
rawdata_comptr_internt.head()
rawdata_comptr_internt.columns

(52, 11)


,OBJECTID,GEO_NAME,FIPS_CODE,Totalhouseholds,Totalhouseholds*marginoferror,TotalHouseholdswithacomputer,TotalHouseholdswithacomputer*marginoferror,TotalHouseholdswithabroadbandInternetsubscription,TotalHouseholdswithabroadbandInternetsubscription*marginoferror,PercentofHouseholdswithInternetAtHome,PercentofHouseholdswithInternetAtHome*marginoferror
0,1,Alabama,1,1860269,6522,1551296,7580,1364318,8150,73.8,0.3
1,2,Alaska,2,253462,1351,236598,1381,213702,1478,84.8,0.4
2,3,Arizona,4,2524300,6840,2269063,8263,2064391,8390,82.2,0.2
3,4,Arkansas,5,1152175,4053,969432,4727,804635,6056,70.4,0.4
4,5,California,6,12965435,19785,11886064,24688,10981568,29085,85.1,0.1


Index(['OBJECTID', 'GEO_NAME', 'FIPS_CODE', 'Totalhouseholds',
       'Totalhouseholds*marginoferror', 'TotalHouseholdswithacomputer',
       'TotalHouseholdswithacomputer*marginoferror',
       'TotalHouseholdswithabroadbandInternetsubscription',
       'TotalHouseholdswithabroadbandInternetsubscription*marginoferror',
       'PercentofHouseholdswithInternetAtHome',
       'PercentofHouseholdswithInternetAtHome*marginoferror'],
      dtype='object')

In [108]:
rawdata_comptr_internt.isnull().sum()

OBJECTID                                                           0
GEO_NAME                                                           0
FIPS_CODE                                                          0
Totalhouseholds                                                    0
Totalhouseholds*marginoferror                                      0
TotalHouseholdswithacomputer                                       0
TotalHouseholdswithacomputer*marginoferror                         0
TotalHouseholdswithabroadbandInternetsubscription                  0
TotalHouseholdswithabroadbandInternetsubscription*marginoferror    0
PercentofHouseholdswithInternetAtHome                              0
PercentofHouseholdswithInternetAtHome*marginoferror                0
dtype: int64

#### Health Insurance Coverage - State

In [156]:
# https://covid19.census.gov/datasets/health-insurance-coverage-counties/data?geometry=138.304%2C-16.868%2C-137.673%2C72.108&page=18
# Health Insurance Coverage - Counties

rawdata_health_ins = pd.read_csv('./Health_Insurance_Coverage_States.csv')
print(rawdata_health_ins.shape)
rawdata_health_ins.columns = [x.lstrip().replace(' ', '') for x in rawdata_health_ins.columns]
rawdata_health_ins.tail()


(52, 83)


,OBJECTID,GEO_ID,GEO_NAME,FIPS_CODE,EmployerInsuranceonly-Pop<19,EmployerInsuranceonly-Pop<19*marginoferror,EmployerInsuranceonly-Pop19-34,EmployerInsuranceonly-Pop19-34*marginoferror,EmployerInsuranceonly-Pop35-64,EmployerInsuranceonly-Pop35-64*marginoferror,EmployerInsuranceonly-Pop65+,EmployerInsuranceonly-Pop65+*marginoferror,DirectPurchaseonly-Pop<19,DirectPurchaseonly-Pop<19*marginoferror,DirectPurchaseonly-Pop19-34,DirectPurchaseonly-Pop19-34*marginoferror,DirectPurchaseonly-Pop35-64,DirectPurchaseonly-Pop35-64*marginoferror,DirectPurchaseonly-Pop65+,DirectPurchaseonly-Pop65+*marginoferror,Medicareonly-Pop<19,Medicareonly-Pop<19*marginoferror,Medicareonly-Pop19-34,Medicareonly-Pop19-34*marginoferror,Medicareonly-Pop35-64,Medicareonly-Pop35-64*marginoferror,Medicareonly-Pop65+,Medicareonly-Pop65+*marginoferror,Medicaidonly-Pop<19,Medicaidonly-Pop<19*marginoferror,Medicaidonly-Pop19-34,Medicaidonly-Pop19-34*marginoferror,Medicaidonly-Pop35-64,Medicaidonly-Pop35-64*marginoferror,Medicaidonly-Pop65+,TRICARE/Militaryonly:-Pop<19,TRICARE/Militaryonly:-Pop<19*marginoferror,TRICARE/Militaryonly:-Pop19-34,TRICARE/Militaryonly:-Pop19-34*marginoferror,TRICARE/Militaryonly:-Pop35-64,TRICARE/Militaryonly:-Pop35-64*marginoferror,TRICARE/Militaryonly:-Pop65+,TRICARE/Militaryonly:-Pop65+*marginoferror,VAHealthCareonly-Pop<19,VAHealthCareonly-Pop<19*marginoferror,VAHealthCareonly-Pop19-34,VAHealthCareonly-Pop19-34*marginoferror,VAHealthCareonly-Pop35-64,VAHealthCareonly-Pop35-64*marginoferror,VAHealthCareonly-Pop65+,VAHealthCareonly-Pop65+*marginoferror,2ormoreInsurance-Pop<19,2ormoreInsurance-Pop<19*marginoferror,2ormoreInsurance-Pop19-34,2ormoreInsurance-Pop19-34*marginoferror,2ormoreInsurance-Pop35-64,2ormoreInsurance-Pop35-64*marginoferror,2ormoreInsurance-Pop65+,2ormoreInsurance-Pop65+*marginoferror,PopulationwithnoHealthInsuranceAges-0-18,PopulationwithnoHealthInsuranceAges-0-18*marginoferror,PopulationwithnoHealthInsuranceAges-19-34,PopulationwithnoHealthInsuranceAges-19-34*marginoferror,PopulationwithnoHealthInsuranceAges-35-64,PopulationwithnoHealthInsuranceAges-35-64*marginoferror,PopulationwithnoHealthInsuranceAges-Age65+,PopulationwithnoHealthInsuranceAges-Age65+*marginoferror,CiviliannoninstitutionalizedpopulationwithHealthCoverage,CiviliannoninstitutionalizedpopulationwithHealthCoverage*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Withhealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulation-Withhealthinsurancecoverage*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Withprivatehealthinsurance,TotalCivilianNoninstitutionalizedPopulation-Withprivatehealthinsurance*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Withpubliccoverage,TotalCivilianNoninstitutionalizedPopulation-Withpubliccoverage*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Nohealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulation-Nohealthinsurancecoverage*marginoferror,Civiliannoninstitutionalizedpopulationunder19years,Civiliannoninstitutionalizedpopulationunder19years*marginoferror,TotalCivilianNoninstitutionalizedPopulationunder19yrs-Nohealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulationunder19yrs-Nohealthinsurancecoverage*marginoferror,PercentofPopulationwithNoHealthInsuranceCoverage,PercentofPopulationwithNoHealthInsuranceCoverage*marginoferror
47,48,0400000US53,Washington,53,828700,7247,892674,5564,1718041,7900,26775,960,99673,3250,123130,2851,242998,3501,4500,488,2738,459,4752,676,38922,1296,267918,3335,570158,7596,241266,3262,271083,4378,NaN,48775,2175,31473,1318,40205,1507,674,210,572,213,5265,562,14175,1004,782,217,118717,3507,101380,2527,258143,3842,746050,3378,53262,2393,198698,4143,229177,4605,6719,683,7187395,1428,6699539,8130,5117530,18086,2500984,12667,487856,8193,1722595,1570,53262,2393,6.8,0.1
48,49,0400000US54,West Virginia,54,179185,3188,180567,2301,394664,3335,4660,415,9795,818,16043,1010,33919,1319,342,90,1069,298,1650,396,25341,1239,81153,1885,164079,3477,815

In [110]:
rawdata_health_ins.isnull().sum()

OBJECTID                                                                                          0
GEO_ID                                                                                            0
GEO_NAME                                                                                          0
FIPS_CODE                                                                                         0
EmployerInsuranceonly-Pop<19                                                                      0
EmployerInsuranceonly-Pop<19*marginoferror                                                        0
EmployerInsuranceonly-Pop19-34                                                                    0
EmployerInsuranceonly-Pop19-34*marginoferror                                                      0
EmployerInsuranceonly-Pop35-64                                                                    0
EmployerInsuranceonly-Pop35-64*marginoferror                                                      0


In [111]:
# Copy Original dataset into new dataframes for further processing
df_comptr_internt = rawdata_comptr_internt.copy()
df_health_ins =  rawdata_health_ins.copy()
df_lang_spkn = rawdata_lang_spkn.copy()
df_popl_povty = rawdata_popl_povty.copy()

In [112]:
# Set index on dataframes
df_comptr_internt.set_index('FIPS_CODE', inplace=True)
df_health_ins.set_index('FIPS_CODE', inplace=True)
df_lang_spkn.set_index('FIPS_CODE', inplace=True)
df_popl_povty.set_index('FIPS_CODE', inplace=True)

In [113]:
df_comptr_internt.head()

,OBJECTID,GEO_NAME,Totalhouseholds,Totalhouseholds*marginoferror,TotalHouseholdswithacomputer,TotalHouseholdswithacomputer*marginoferror,TotalHouseholdswithabroadbandInternetsubscription,TotalHouseholdswithabroadbandInternetsubscription*marginoferror,PercentofHouseholdswithInternetAtHome,PercentofHouseholdswithInternetAtHome*marginoferror
FIPS_CODE,,,,,,,,,,
1,1,Alabama,1860269,6522,1551296,7580,1364318,8150,73.8,0.3
2,2,Alaska,253462,1351,236598,1381,213702,1478,84.8,0.4
4,3,Arizona,2524300,6840,2269063,8263,2064391,8390,82.2,0.2
5,4,Arkansas,1152175,4053,969432,4727,804635,6056,70.4,0.4
6,5,California,12965435,19785,11886064,24688,10981568,29085,85.1,0.1


In [114]:
df_state_census = df_health_ins.join(df_comptr_internt, on='FIPS_CODE', how='inner', rsuffix='_other')
df_state_census = df_state_census.join(df_lang_spkn, on='FIPS_CODE', how='inner', rsuffix='_other')
df_state_census = df_state_census.join(df_popl_povty, on='FIPS_CODE', how='inner', rsuffix='_other')
df_state_census.reset_index(inplace=True)
df_state_census.tail()

,FIPS_CODE,OBJECTID,GEO_ID,GEO_NAME,EmployerInsuranceonly-Pop<19,EmployerInsuranceonly-Pop<19*marginoferror,EmployerInsuranceonly-Pop19-34,EmployerInsuranceonly-Pop19-34*marginoferror,EmployerInsuranceonly-Pop35-64,EmployerInsuranceonly-Pop35-64*marginoferror,EmployerInsuranceonly-Pop65+,EmployerInsuranceonly-Pop65+*marginoferror,DirectPurchaseonly-Pop<19,DirectPurchaseonly-Pop<19*marginoferror,DirectPurchaseonly-Pop19-34,DirectPurchaseonly-Pop19-34*marginoferror,DirectPurchaseonly-Pop35-64,DirectPurchaseonly-Pop35-64*marginoferror,DirectPurchaseonly-Pop65+,DirectPurchaseonly-Pop65+*marginoferror,Medicareonly-Pop<19,Medicareonly-Pop<19*marginoferror,Medicareonly-Pop19-34,Medicareonly-Pop19-34*marginoferror,Medicareonly-Pop35-64,Medicareonly-Pop35-64*marginoferror,Medicareonly-Pop65+,Medicareonly-Pop65+*marginoferror,Medicaidonly-Pop<19,Medicaidonly-Pop<19*marginoferror,Medicaidonly-Pop19-34,Medicaidonly-Pop19-34*marginoferror,Medicaidonly-Pop35-64,Medicaidonly-Pop35-64*marginoferror,Medicaidonly-Pop65+,TRICARE/Militaryonly:-Pop<19,TRICARE/Militaryonly:-Pop<19*marginoferror,TRICARE/Militaryonly:-Pop19-34,TRICARE/Militaryonly:-Pop19-34*marginoferror,TRICARE/Militaryonly:-Pop35-64,TRICARE/Militaryonly:-Pop35-64*marginoferror,TRICARE/Militaryonly:-Pop65+,TRICARE/Militaryonly:-Pop65+*marginoferror,VAHealthCareonly-Pop<19,VAHealthCareonly-Pop<19*marginoferror,VAHealthCareonly-Pop19-34,VAHealthCareonly-Pop19-34*marginoferror,VAHealthCareonly-Pop35-64,VAHealthCareonly-Pop35-64*marginoferror,VAHealthCareonly-Pop65+,VAHealthCareonly-Pop65+*marginoferror,2ormoreInsurance-Pop<19,2ormoreInsurance-Pop<19*marginoferror,2ormoreInsurance-Pop19-34,2ormoreInsurance-Pop19-34*marginoferror,2ormoreInsurance-Pop35-64,2ormoreInsurance-Pop35-64*marginoferror,2ormoreInsurance-Pop65+,2ormoreInsurance-Pop65+*marginoferror,PopulationwithnoHealthInsuranceAges-0-18,PopulationwithnoHealthInsuranceAges-0-18*marginoferror,PopulationwithnoHealthInsuranceAges-19-34,PopulationwithnoHealthInsuranceAges-19-34*marginoferror,PopulationwithnoHealthInsuranceAges-35-64,PopulationwithnoHealthInsuranceAges-35-64*marginoferror,PopulationwithnoHealthInsuranceAges-Age65+,PopulationwithnoHealthInsuranceAges-Age65+*marginoferror,CiviliannoninstitutionalizedpopulationwithHealthCoverage,CiviliannoninstitutionalizedpopulationwithHealthCoverage*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Withhealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulation-Withhealthinsurancecoverage*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Withprivatehealthinsurance,TotalCivilianNoninstitutionalizedPopulation-Withprivatehealthinsurance*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Withpubliccoverage,TotalCivilianNoninstitutionalizedPopulation-Withpubliccoverage*marginoferror,TotalCivilianNoninstitutionalizedPopulation-Nohealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulation-Nohealthinsurancecoverage*marginoferror,Civiliannoninstitutionalizedpopulationunder19years,Civiliannoninstitutionalizedpopulationunder19years*marginoferror,TotalCivilianNoninstitutionalizedPopulationunder19yrs-Nohealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulationunder19yrs-Nohealthinsurancecoverage*marginoferror,PercentofPopulationwithNoHealthInsuranceCoverage,PercentofPopulationwithNoHealthInsuranceCoverage*marginoferror,OBJECTID_other,GEO_NAME_other,Totalhouseholds,Totalhouseholds*marginoferror,TotalHouseholdswithacomputer,TotalHouseholdswithacomputer*marginoferror,TotalHouseholdswithabroadbandInternetsubscription,TotalHouseholdswithabroadbandInternetsubscription*marginoferror,PercentofHouseholdswithInternetAtHome,PercentofHouseholdswithInternetAtHome*marginoferror,OBJECTID_other,GEO_ID_other,GEO_NAME_other,EnglishOnly-Pop5-17,EnglishOnly-Pop5-17*marginoferror,EnglishOnly-Pop18-64,EnglishOnly-Pop18-64*marginoferror,EnglishOnly-Pop65+,EnglishOnly-Pop65+*marginoferror,"LanguageSpokenatHomebyAge-Spanish-SpeakEnglish""lessthanverywell""5andover","LanguageS

In [115]:
# Do not remove any rows for state/county, instead update missing values with 0 for further processing
df_state_census = df_state_census.fillna(0)

In [116]:
# Check for NANs in dataframe
df_state_census.isnull().sum()

FIPS_CODE                                                    0
OBJECTID                                                     0
GEO_ID                                                       0
GEO_NAME                                                     0
EmployerInsuranceonly-Pop<19                                 0
                                                            ..
TotalMarried-CoupleFamilyHouseholds*marginoferror            0
TotalOtherFamiliesHHldsw/MaleHouseholder                     0
TotalOtherFamiliesHHldsw/MaleHouseholder*marginoferror       0
TotalOtherFamiliesHhldsw/FemaleHousehiolder                  0
TotalOtherFamiliesHhldsw/FemaleHousehiolder*marginoferror    0
Length: 274, dtype: int64

In [117]:
# Check for NANs in dataframe
df_state_census.isnull().sum()

FIPS_CODE                                                    0
OBJECTID                                                     0
GEO_ID                                                       0
GEO_NAME                                                     0
EmployerInsuranceonly-Pop<19                                 0
                                                            ..
TotalMarried-CoupleFamilyHouseholds*marginoferror            0
TotalOtherFamiliesHHldsw/MaleHouseholder                     0
TotalOtherFamiliesHHldsw/MaleHouseholder*marginoferror       0
TotalOtherFamiliesHhldsw/FemaleHousehiolder                  0
TotalOtherFamiliesHhldsw/FemaleHousehiolder*marginoferror    0
Length: 274, dtype: int64

In [118]:
list(df_state_census.columns)

['FIPS_CODE',
 'OBJECTID',
 'GEO_ID',
 'GEO_NAME',
 'EmployerInsuranceonly-Pop<19',
 'EmployerInsuranceonly-Pop<19*marginoferror',
 'EmployerInsuranceonly-Pop19-34',
 'EmployerInsuranceonly-Pop19-34*marginoferror',
 'EmployerInsuranceonly-Pop35-64',
 'EmployerInsuranceonly-Pop35-64*marginoferror',
 'EmployerInsuranceonly-Pop65+',
 'EmployerInsuranceonly-Pop65+*marginoferror',
 'DirectPurchaseonly-Pop<19',
 'DirectPurchaseonly-Pop<19*marginoferror',
 'DirectPurchaseonly-Pop19-34',
 'DirectPurchaseonly-Pop19-34*marginoferror',
 'DirectPurchaseonly-Pop35-64',
 'DirectPurchaseonly-Pop35-64*marginoferror',
 'DirectPurchaseonly-Pop65+',
 'DirectPurchaseonly-Pop65+*marginoferror',
 'Medicareonly-Pop<19',
 'Medicareonly-Pop<19*marginoferror',
 'Medicareonly-Pop19-34',
 'Medicareonly-Pop19-34*marginoferror',
 'Medicareonly-Pop35-64',
 'Medicareonly-Pop35-64*marginoferror',
 'Medicareonly-Pop65+',
 'Medicareonly-Pop65+*marginoferror',
 'Medicaidonly-Pop<19',
 'Medicaidonly-Pop<19*marginoferror',

#### List of columns to drop

In [119]:
drop_cols = pd.read_csv('./df_census_ONLY_exportdata-STATE-LEVEL-DROPCOLS.csv')
print(drop_cols.shape)
drop_cols.head()
drop_cols.set_index('columns_to_drop', inplace=True)
list_columns_to_drop = list(drop_cols.index)
list_columns_to_drop[:10]

(127, 1)


,columns_to_drop
0,EmployerInsuranceonly-Pop<19*marginoferror
1,EmployerInsuranceonly-Pop19-34*marginoferror
2,EmployerInsuranceonly-Pop35-64*marginoferror
3,EmployerInsuranceonly-Pop65+*marginoferror
4,DirectPurchaseonly-Pop<19*marginoferror


['EmployerInsuranceonly-Pop<19*marginoferror',
 'EmployerInsuranceonly-Pop19-34*marginoferror',
 'EmployerInsuranceonly-Pop35-64*marginoferror',
 'EmployerInsuranceonly-Pop65+*marginoferror',
 'DirectPurchaseonly-Pop<19*marginoferror',
 'DirectPurchaseonly-Pop19-34*marginoferror',
 'DirectPurchaseonly-Pop35-64*marginoferror',
 'DirectPurchaseonly-Pop65+*marginoferror',
 'Medicareonly-Pop<19*marginoferror',
 'Medicareonly-Pop19-34*marginoferror']

In [120]:
# Drop columns created as part of joining the dataframe(s) for further processing
df_state_census = df_state_census.drop(list_columns_to_drop, axis=1)

In [121]:
print(df_state_census.shape)

(52, 147)


In [122]:
# Create dataframe for GEO information to track information about each observation row
fips_code = df_state_census['FIPS_CODE']
fips_code.head()

0    1
1    2
2    4
3    5
4    6
Name: FIPS_CODE, dtype: int64

In [123]:
# Drop unique identifiers and GEO name columns from the dataframe for further processing
df_state_census = df_state_census.drop(['OBJECTID','GEO_ID','GEO_NAME','GEO_ID_other','GEO_NAME_other','OBJECTID_other'], axis=1)

In [124]:
df_state_census.shape

(52, 136)

In [125]:
# Perform one-hot encoding (ohe) on the catergorical variables in the dataset
df_state_census = pd.get_dummies(df_state_census, drop_first=True)
df_state_census.tail()

,FIPS_CODE,EmployerInsuranceonly-Pop<19,EmployerInsuranceonly-Pop19-34,EmployerInsuranceonly-Pop35-64,EmployerInsuranceonly-Pop65+,DirectPurchaseonly-Pop<19,DirectPurchaseonly-Pop19-34,DirectPurchaseonly-Pop35-64,DirectPurchaseonly-Pop65+,Medicareonly-Pop<19,Medicareonly-Pop19-34,Medicareonly-Pop35-64,Medicareonly-Pop65+,Medicaidonly-Pop<19,Medicaidonly-Pop19-34,Medicaidonly-Pop35-64,Medicaidonly-Pop65+,TRICARE/Militaryonly:-Pop<19,TRICARE/Militaryonly:-Pop19-34,TRICARE/Militaryonly:-Pop35-64,TRICARE/Militaryonly:-Pop65+,VAHealthCareonly-Pop<19,VAHealthCareonly-Pop19-34,VAHealthCareonly-Pop35-64,VAHealthCareonly-Pop65+,2ormoreInsurance-Pop<19,2ormoreInsurance-Pop19-34,2ormoreInsurance-Pop35-64,2ormoreInsurance-Pop65+,PopulationwithnoHealthInsuranceAges-0-18,PopulationwithnoHealthInsuranceAges-19-34,PopulationwithnoHealthInsuranceAges-35-64,PopulationwithnoHealthInsuranceAges-Age65+,CiviliannoninstitutionalizedpopulationwithHealthCoverage,TotalCivilianNoninstitutionalizedPopulation-Withhealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulation-Withprivatehealthinsurance,TotalCivilianNoninstitutionalizedPopulation-Withpubliccoverage,TotalCivilianNoninstitutionalizedPopulation-Nohealthinsurancecoverage,Civiliannoninstitutionalizedpopulationunder19years,TotalCivilianNoninstitutionalizedPopulationunder19yrs-Nohealthinsurancecoverage,PercentofPopulationwithNoHealthInsuranceCoverage,Totalhouseholds,TotalHouseholdswithacomputer,TotalHouseholdswithabroadbandInternetsubscription,PercentofHouseholdswithInternetAtHome,EnglishOnly-Pop5-17,EnglishOnly-Pop18-64,EnglishOnly-Pop65+,"LanguageSpokenatHomebyAge-Spanish-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherIndo-European-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-AsianPacificIslands-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherLanguages-SpeakEnglish""lessthanverywell""5andover",LanguageSpokenatHome-Spanish(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-OtherIndo-European(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-AsianandPacificIsland(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-Other(LimitedEnglishHHs/allLimitedEnglishHHs),EnglishOnly-Total,Spanish-Pop5-17,Spanish-Pop18-64,Spanish-Pop65+,Spanish-Total,Spanish-SpeakEnglishverywell-Pop5-17,Spanish-SpeakEnglishverywell-Pop18-64,Spanish-SpeakEnglishverywell-Pop65+,Spanish-SpeakEnglishverywell-Total,Spanish-Englishlessthanverywell-Pop5-17,Spanish-Englishlessthanverywell-Pop18-64,Spanish-Englishlessthanverywell-Pop65+,Spanish-Englishlessthanverywell-Total,OtherIndo-European-Pop5-17,OtherIndo-European-Pop18-64,OtherIndo-European-Pop65+,OtherIndo-European-Total,OtherIndo-European-SpeakEnglishverywell-Pop5-17,OtherIndo-European-SpeakEnglishverywell-Pop18-64,OtherIndo-European-SpeakEnglishverywell-Pop65+,OtherIndo-European-SpeakEnglishverywell-Total,OtherIndo-European-Englishlessthanverywell-Pop5-17,OtherIndo-European-Englishlessthanverywell-Pop18-64,OtherIndo-European-Englishlessthanverywell-Pop65+,OtherIndo-European-Englishlessthanverywell-Total,Asian-PacificIsland-Pop5-17,Asian-PacificIsland-Pop18-64,Asian-PacificIsland-Pop65+,Asian-PacificIsland-Total,Asian-PacificIsland-SpeakEnglishverywell-Pop5-17,Asian-PacificIsland-SpeakEnglishverywell-Pop18-64,Asian-PacificIsland-SpeakEnglishverywell-Pop65+,Asian-PacificIsland-SpeakEnglishverywell-Total,Asian-PacificIsland-Englishlessthanverywell-Pop5-17,Asian-PacificIsland-Englishlessthanverywell-Pop18-64,Asian-PacificIsland-Englishlessthanverywell-Pop65+,Asian-PacificIsland-Englishlessthanverywell-Total,OtherLanguages-Pop5-17,OtherLanguages-Pop18-64,OtherLanguages-Pop65+,OtherLanguages-Total,OtherLanguages-SpeakEnglishverywell-Pop5-17,OtherLanguages-SpeakEnglishverywell-Pop18-64,OtherLanguages-SpeakEnglishverywell-Pop65+,OtherLanguages-SpeakEnglishverywell-Total,OtherLanguages-Englishlessthanverywell-Pop5-17,OtherLanguages-Englishlessthanverywell-Pop18-64,OtherLangua

In [126]:
list(df_state_census.columns)

['FIPS_CODE',
 'EmployerInsuranceonly-Pop<19',
 'EmployerInsuranceonly-Pop19-34',
 'EmployerInsuranceonly-Pop35-64',
 'EmployerInsuranceonly-Pop65+',
 'DirectPurchaseonly-Pop<19',
 'DirectPurchaseonly-Pop19-34',
 'DirectPurchaseonly-Pop35-64',
 'DirectPurchaseonly-Pop65+',
 'Medicareonly-Pop<19',
 'Medicareonly-Pop19-34',
 'Medicareonly-Pop35-64',
 'Medicareonly-Pop65+',
 'Medicaidonly-Pop<19',
 'Medicaidonly-Pop19-34',
 'Medicaidonly-Pop35-64',
 'Medicaidonly-Pop65+',
 'TRICARE/Militaryonly:-Pop<19',
 'TRICARE/Militaryonly:-Pop19-34',
 'TRICARE/Militaryonly:-Pop35-64',
 'TRICARE/Militaryonly:-Pop65+',
 'VAHealthCareonly-Pop<19',
 'VAHealthCareonly-Pop19-34',
 'VAHealthCareonly-Pop35-64',
 'VAHealthCareonly-Pop65+',
 '2ormoreInsurance-Pop<19',
 '2ormoreInsurance-Pop19-34',
 '2ormoreInsurance-Pop35-64',
 '2ormoreInsurance-Pop65+',
 'PopulationwithnoHealthInsuranceAges-0-18',
 'PopulationwithnoHealthInsuranceAges-19-34',
 'PopulationwithnoHealthInsuranceAges-35-64',
 'PopulationwithnoHea

In [127]:
# Save dataframe to .csv file
df_state_census.to_csv('./df_census_ONLY_exportdata-STATE-LEVEL-aFE.csv')

# Merging Census Data 

In [128]:
census_data = df_state_census
census_data.tail()
census_data.shape

,FIPS_CODE,EmployerInsuranceonly-Pop<19,EmployerInsuranceonly-Pop19-34,EmployerInsuranceonly-Pop35-64,EmployerInsuranceonly-Pop65+,DirectPurchaseonly-Pop<19,DirectPurchaseonly-Pop19-34,DirectPurchaseonly-Pop35-64,DirectPurchaseonly-Pop65+,Medicareonly-Pop<19,Medicareonly-Pop19-34,Medicareonly-Pop35-64,Medicareonly-Pop65+,Medicaidonly-Pop<19,Medicaidonly-Pop19-34,Medicaidonly-Pop35-64,Medicaidonly-Pop65+,TRICARE/Militaryonly:-Pop<19,TRICARE/Militaryonly:-Pop19-34,TRICARE/Militaryonly:-Pop35-64,TRICARE/Militaryonly:-Pop65+,VAHealthCareonly-Pop<19,VAHealthCareonly-Pop19-34,VAHealthCareonly-Pop35-64,VAHealthCareonly-Pop65+,2ormoreInsurance-Pop<19,2ormoreInsurance-Pop19-34,2ormoreInsurance-Pop35-64,2ormoreInsurance-Pop65+,PopulationwithnoHealthInsuranceAges-0-18,PopulationwithnoHealthInsuranceAges-19-34,PopulationwithnoHealthInsuranceAges-35-64,PopulationwithnoHealthInsuranceAges-Age65+,CiviliannoninstitutionalizedpopulationwithHealthCoverage,TotalCivilianNoninstitutionalizedPopulation-Withhealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulation-Withprivatehealthinsurance,TotalCivilianNoninstitutionalizedPopulation-Withpubliccoverage,TotalCivilianNoninstitutionalizedPopulation-Nohealthinsurancecoverage,Civiliannoninstitutionalizedpopulationunder19years,TotalCivilianNoninstitutionalizedPopulationunder19yrs-Nohealthinsurancecoverage,PercentofPopulationwithNoHealthInsuranceCoverage,Totalhouseholds,TotalHouseholdswithacomputer,TotalHouseholdswithabroadbandInternetsubscription,PercentofHouseholdswithInternetAtHome,EnglishOnly-Pop5-17,EnglishOnly-Pop18-64,EnglishOnly-Pop65+,"LanguageSpokenatHomebyAge-Spanish-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherIndo-European-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-AsianPacificIslands-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherLanguages-SpeakEnglish""lessthanverywell""5andover",LanguageSpokenatHome-Spanish(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-OtherIndo-European(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-AsianandPacificIsland(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-Other(LimitedEnglishHHs/allLimitedEnglishHHs),EnglishOnly-Total,Spanish-Pop5-17,Spanish-Pop18-64,Spanish-Pop65+,Spanish-Total,Spanish-SpeakEnglishverywell-Pop5-17,Spanish-SpeakEnglishverywell-Pop18-64,Spanish-SpeakEnglishverywell-Pop65+,Spanish-SpeakEnglishverywell-Total,Spanish-Englishlessthanverywell-Pop5-17,Spanish-Englishlessthanverywell-Pop18-64,Spanish-Englishlessthanverywell-Pop65+,Spanish-Englishlessthanverywell-Total,OtherIndo-European-Pop5-17,OtherIndo-European-Pop18-64,OtherIndo-European-Pop65+,OtherIndo-European-Total,OtherIndo-European-SpeakEnglishverywell-Pop5-17,OtherIndo-European-SpeakEnglishverywell-Pop18-64,OtherIndo-European-SpeakEnglishverywell-Pop65+,OtherIndo-European-SpeakEnglishverywell-Total,OtherIndo-European-Englishlessthanverywell-Pop5-17,OtherIndo-European-Englishlessthanverywell-Pop18-64,OtherIndo-European-Englishlessthanverywell-Pop65+,OtherIndo-European-Englishlessthanverywell-Total,Asian-PacificIsland-Pop5-17,Asian-PacificIsland-Pop18-64,Asian-PacificIsland-Pop65+,Asian-PacificIsland-Total,Asian-PacificIsland-SpeakEnglishverywell-Pop5-17,Asian-PacificIsland-SpeakEnglishverywell-Pop18-64,Asian-PacificIsland-SpeakEnglishverywell-Pop65+,Asian-PacificIsland-SpeakEnglishverywell-Total,Asian-PacificIsland-Englishlessthanverywell-Pop5-17,Asian-PacificIsland-Englishlessthanverywell-Pop18-64,Asian-PacificIsland-Englishlessthanverywell-Pop65+,Asian-PacificIsland-Englishlessthanverywell-Total,OtherLanguages-Pop5-17,OtherLanguages-Pop18-64,OtherLanguages-Pop65+,OtherLanguages-Total,OtherLanguages-SpeakEnglishverywell-Pop5-17,OtherLanguages-SpeakEnglishverywell-Pop18-64,OtherLanguages-SpeakEnglishverywell-Pop65+,OtherLanguages-SpeakEnglishverywell-Total,OtherLanguages-Englishlessthanverywell-Pop5-17,OtherLanguages-Englishlessthanverywell-Pop18-64,OtherLangua

(52, 136)

In [158]:
# checking if we have similar states between the two datasets
different_c= list(set(census_data['FIPS_CODE'])- set(policies_race['State FIPS Code']))
different_c

[72]

In [159]:
census_data.drop(index =51, axis=0, inplace =True)

In [160]:
census_data.tail()
census_data.shape

,FIPS_CODE,EmployerInsuranceonly-Pop<19,EmployerInsuranceonly-Pop19-34,EmployerInsuranceonly-Pop35-64,EmployerInsuranceonly-Pop65+,DirectPurchaseonly-Pop<19,DirectPurchaseonly-Pop19-34,DirectPurchaseonly-Pop35-64,DirectPurchaseonly-Pop65+,Medicareonly-Pop<19,Medicareonly-Pop19-34,Medicareonly-Pop35-64,Medicareonly-Pop65+,Medicaidonly-Pop<19,Medicaidonly-Pop19-34,Medicaidonly-Pop35-64,Medicaidonly-Pop65+,TRICARE/Militaryonly:-Pop<19,TRICARE/Militaryonly:-Pop19-34,TRICARE/Militaryonly:-Pop35-64,TRICARE/Militaryonly:-Pop65+,VAHealthCareonly-Pop<19,VAHealthCareonly-Pop19-34,VAHealthCareonly-Pop35-64,VAHealthCareonly-Pop65+,2ormoreInsurance-Pop<19,2ormoreInsurance-Pop19-34,2ormoreInsurance-Pop35-64,2ormoreInsurance-Pop65+,PopulationwithnoHealthInsuranceAges-0-18,PopulationwithnoHealthInsuranceAges-19-34,PopulationwithnoHealthInsuranceAges-35-64,PopulationwithnoHealthInsuranceAges-Age65+,CiviliannoninstitutionalizedpopulationwithHealthCoverage,TotalCivilianNoninstitutionalizedPopulation-Withhealthinsurancecoverage,TotalCivilianNoninstitutionalizedPopulation-Withprivatehealthinsurance,TotalCivilianNoninstitutionalizedPopulation-Withpubliccoverage,TotalCivilianNoninstitutionalizedPopulation-Nohealthinsurancecoverage,Civiliannoninstitutionalizedpopulationunder19years,TotalCivilianNoninstitutionalizedPopulationunder19yrs-Nohealthinsurancecoverage,PercentofPopulationwithNoHealthInsuranceCoverage,Totalhouseholds,TotalHouseholdswithacomputer,TotalHouseholdswithabroadbandInternetsubscription,PercentofHouseholdswithInternetAtHome,EnglishOnly-Pop5-17,EnglishOnly-Pop18-64,EnglishOnly-Pop65+,"LanguageSpokenatHomebyAge-Spanish-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherIndo-European-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-AsianPacificIslands-SpeakEnglish""lessthanverywell""5andover","LanguageSpokenatHomebyAge-OtherLanguages-SpeakEnglish""lessthanverywell""5andover",LanguageSpokenatHome-Spanish(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-OtherIndo-European(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-AsianandPacificIsland(LimitedEnglishHHs/allLimitedEnglishHHs),LanguageSpokenatHome-Other(LimitedEnglishHHs/allLimitedEnglishHHs),EnglishOnly-Total,Spanish-Pop5-17,Spanish-Pop18-64,Spanish-Pop65+,Spanish-Total,Spanish-SpeakEnglishverywell-Pop5-17,Spanish-SpeakEnglishverywell-Pop18-64,Spanish-SpeakEnglishverywell-Pop65+,Spanish-SpeakEnglishverywell-Total,Spanish-Englishlessthanverywell-Pop5-17,Spanish-Englishlessthanverywell-Pop18-64,Spanish-Englishlessthanverywell-Pop65+,Spanish-Englishlessthanverywell-Total,OtherIndo-European-Pop5-17,OtherIndo-European-Pop18-64,OtherIndo-European-Pop65+,OtherIndo-European-Total,OtherIndo-European-SpeakEnglishverywell-Pop5-17,OtherIndo-European-SpeakEnglishverywell-Pop18-64,OtherIndo-European-SpeakEnglishverywell-Pop65+,OtherIndo-European-SpeakEnglishverywell-Total,OtherIndo-European-Englishlessthanverywell-Pop5-17,OtherIndo-European-Englishlessthanverywell-Pop18-64,OtherIndo-European-Englishlessthanverywell-Pop65+,OtherIndo-European-Englishlessthanverywell-Total,Asian-PacificIsland-Pop5-17,Asian-PacificIsland-Pop18-64,Asian-PacificIsland-Pop65+,Asian-PacificIsland-Total,Asian-PacificIsland-SpeakEnglishverywell-Pop5-17,Asian-PacificIsland-SpeakEnglishverywell-Pop18-64,Asian-PacificIsland-SpeakEnglishverywell-Pop65+,Asian-PacificIsland-SpeakEnglishverywell-Total,Asian-PacificIsland-Englishlessthanverywell-Pop5-17,Asian-PacificIsland-Englishlessthanverywell-Pop18-64,Asian-PacificIsland-Englishlessthanverywell-Pop65+,Asian-PacificIsland-Englishlessthanverywell-Total,OtherLanguages-Pop5-17,OtherLanguages-Pop18-64,OtherLanguages-Pop65+,OtherLanguages-Total,OtherLanguages-SpeakEnglishverywell-Pop5-17,OtherLanguages-SpeakEnglishverywell-Pop18-64,OtherLanguages-SpeakEnglishverywell-Pop65+,OtherLanguages-SpeakEnglishverywell-Total,OtherLanguages-Englishlessthanverywell-Pop5-17,OtherLanguages-Englishlessthanverywell-Pop18-64,OtherLangua

(51, 136)

In [132]:
policy_race_census = pd.merge(policies_race, census_data,left_on='State FIPS Code',
     right_on='FIPS_CODE',how='left')
policy_race_census.head()
policy_race_census.shape
policy_race_census.info()

,State_x,State Abbreviation,State FIPS Code,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Closed Bars,State of emergency,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Begin to Re-Close Bars,Re-Close Bars (statewide),Re-Close Movie Theaters (statewide),Re-Close Gyms (statewide),Re-Close Indoor Dining (Statewide),Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Alcohol/Liquor Stores Open,Allow restaurants to sell takeout alcohol,Allow restaurants to deliver alcohol,Keep Firearms Sellers Open,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,State had CHIP premium non-payment lock-out period as of January 2019,Supend CHIP premium non-payment lock-outs,Suspended elective medical/dental procedures,Made Effort to Limit Abortion Access,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Does not charge copays for incarcerated individuals,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Did not waive copays for incarcerated individuals,Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing,Patients can receive 14-28 take-home doses of opioid medication,Home delivery of take-home medicati

(51, 256)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Columns: 256 entries, State_x to TotalOtherFamiliesHhldsw/FemaleHousehiolder
dtypes: datetime64[ns](1), float64(104), int64(147), object(4)
memory usage: 102.4+ KB


In [133]:
drop_list_final=['Date closed K-12 schools'
,'Date banned visitors to nursing homes'
,'Closed restaurants except take out'
,'Closed gyms'
,'Closed movie theaters'
,'Closed Bars'
,'Begin to Re-Close Bars'
,'Re-Close Bars (statewide)'
,'Re-Close Movie Theaters (statewide)'
,'Re-Close Gyms (statewide)'
,'Re-Close Indoor Dining (Statewide)'
,'Alcohol/Liquor Stores Open'
,'Allow restaurants to deliver alcohol'
,'Keep Firearms Sellers Open'
,'State had CHIP premium non-payment lock-out period as of January 2019'
,'Supend CHIP premium non-payment lock-outs'
,'Suspended elective medical/dental procedures'
,'Made Effort to Limit Abortion Access'
,'Does not charge copays for incarcerated individuals'
,'Did not waive copays for incarcerated individuals'
,'Use of telemedicine/telephone evaluations to initiate buprenorphine prescribing'
,'Patients can receive 14-28 take-home doses of opioid medication'
,'Home delivery of take-home medication by opioid treatment programs'
,'Use of telemedicine for schedule II-V prescriptions'
,'Exceptions to emergency oral prescriptions'
,'Waive requirement to obtain separate DEA registration to dispense outside home state',
'No legal enforcement of face mask mandate','State_y','State FIPS Code']

In [134]:
policy_race_census_master =policy_race_census.drop(drop_list_final, axis=1)
policy_race_census_master.head()

,State_x,State Abbreviation,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,Closed day cares,Closed non-essential businesses,State of emergency,Stay at home/ shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,End/relax stay at home/shelter in place,Began to reopen businesses statewide,Reopen restaurants,Initially reopen restaurants for outdoor dining only,Reopened gyms,Reopened movie theaters,Reopened hair salons/barber shops,Reopen non-essential construction,Reopen Religious gatherings,Reopen non-essential retail,Reopen bars,Reopen Childcare,Resumed elective medical procedures,Mandate face mask use by all individuals in public spaces,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,Mandate face mask use by employees in public-facing businesses,Mandate quarantine for those entering the state from specific states,Mandate quarantine for all individuals entering the state from another state,Allow restaurants to sell takeout alcohol,Stop Initiation of Evictions overall or due to COVID related issues,Stop enforcement of evictions overall or due to COVID related issues,Renter grace period or use of security deposit to pay rent,Order freezing utility shut offs,Froze mortgage payments,Lift Eviction Moratorium,"Prior to pandemic, no state unemployment waiting period; or date waiting period waived not found",Waived one week waiting period for unemployment insurance,Waive work search requirement for unemployment insurance,Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined,Expand eligibility to high-risk individuals in preventative quarantine,Expand eligibility of unemployment insurance to those who have lost childcare/school closures,Extend the amount of time an individual can be on unemployment insurance,Weekly unemployment insurance maximum amount (dollars),"Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)",Unemployment insurance maximum duration (weeks),Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks),SNAP Waiver-Emergency Allotments to Current SNAP Households,SNAP Waiver-Pandemic EBT,SNAP Waiver-Temporary Suspension of Claims Collection,2020 Q1 SNAP ABAWD Time Limit Waiver,Modify Medicaid requirements with 1135 waivers (date of CMS approval),Reopened ACA enrollment using a special enrollment period,State previously allowed audio-only telehealth,Allow audio-only telehealth,Allow/expand Medicaid telehealth coverage,Stopped personal visitation in state prisons,Stopped in-person attorney visits in state prisons,Waived COVID/respiratory illness-related copays during pandemic for incarcerated individuals,Waived all copays during pandemic for incarcerated individuals,Cases_Total,Deaths_Total,Cases_White_percent,Cases_Black_percent,Cases_LatinX_percent,Cases_Asian_percent,Cases_AIAN_percent,Cases_NHPI_percent,Cases_Multiracial_percent,Cases_Other_percent,Cases_Unknown_percent,Deaths_White_percent,Deaths_Black_percent,DeathsDeaths_LatinX_percent,Deaths_Asian_percent,Deaths_AIAN_percent,Deaths_NHPI_percent,Deaths_Multiracial_percent,Deaths_Other_percent,Deaths_Unknown_percent,Cases_Total_percent,Deaths_Total_percent,homeles_percent,All-cause deaths 2018_percent,FIPS_CODE,EmployerInsuranceonly-Pop<19,EmployerInsuranceonly-Pop19-34,EmployerInsuranceonly-Pop35-64,EmployerInsuranceonly-Pop65+,DirectPurchaseonly-Pop<19,DirectPurchaseonly-Pop19-34,DirectPurchaseonly-Pop35-64,DirectPurchaseonly-Pop65+,Medicareonly-Pop<19,Medicareonly-Pop19-34,Medicareonly-Pop35-64,Medicareonly-Pop65+,Medicaidonly-Pop<19,Medicaidonly-Pop19-34,Medicaidonly-Pop35-64,Medicaidonly-Pop65+,TRICARE

In [135]:
policy_race_census_master.to_csv('policy_race_census_master')

#### The master list is the combination of 4 different datasets and will be used for the machine learning analysis.